# Library of the Haskala
Converts from msql to SQlite for Django project
Converts from mysql to Turtle for including the data in the Graph.


In [2]:
import os
import mysql
import pandas as pd
import datetime
import gzip
import math
import shutil
import environ
from datetime import datetime

from rdflib import Graph, Namespace, RDF, Literal, URIRef

In [3]:
# Read .env file
env = environ.Env(
    # set casting, default value
    DEBUG=(bool, False)
)

# Take environment variables from .env file
environ.Env.read_env('.env')

DB_USER = env('DB_USER')
DB_PASSWORD = env('DB_PASSWORD')
DB_HOST = env('DB_HOST')
DB_NAME = env('DB_NAME')

In [4]:
from sqlalchemy import create_engine
import pandas as pd

db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
db_connection = create_engine(db_connection_str)

In [5]:
haskala_node = pd.read_sql('SELECT * FROM node', con=db_connection)
haskala_field_config = pd.read_sql('SELECT * FROM field_config', con=db_connection)
haskala_node.head(5)

,nid,vid,type,language,title,uid,status,created,changed,comment,promote,sticky,tnid,translate
0,1,1,page_element,en,Introduction,1,1,1428490114,1428490114,0,0,0,0,0
1,2,2,person,und,(Dr. med.) איסטרייכער,1,1,1428490114,1428490114,0,1,0,0,0
2,3,3,person,und,(Dr. med.) ווארבורג,1,1,1428490114,1428490114,0,1,0,0,0
3,4,4,person,und,(Dr.) בריינערסדארף,1,1,1428490114,1428490114,0,1,0,0,0
4,5,5,person,und,"(Feder)-Guttmann, Tuvia - פדר, טוביה",1,1,1428490114,1428490114,0,1,0,0,0


In [6]:
print(haskala_node['type'].value_counts())

type
person          1980
mention         1867
production       667
book             525
edition          327
preface          272
translation       19
page_element       1
Name: count, dtype: int64


In [7]:
print(haskala_node['language'].value_counts())

language
und    5657
en        1
Name: count, dtype: int64


In [8]:
persons = haskala_node[haskala_node['type'] == 'person']['title']
persons_list = persons.to_list()
persons

1                             (Dr. med.) איסטרייכער
2                               (Dr. med.) ווארבורג
3                                (Dr.) בריינערסדארף
4              (Feder)-Guttmann, Tuvia - פדר, טוביה
5                Aaron, Joseph Philipp - אהרן, יוסף
                           ...                     
5579                                       י"ט בנעט
5585                                    משה זאללשין
5594    הילל ברי' דרב שמואל ספרא דק"ק טיקטין והגליל
5597                        אבן תבון, יהודה בן שאול
5607                                         הרש"בם
Name: title, Length: 1980, dtype: object

In [9]:
persons.count()

np.int64(1980)

In [10]:
mentions = haskala_node[haskala_node['type'] == 'mention']['title']
mentions.to_list()

["Alexander, Levy - Answer to Mr. Joshua Van Oven’s letters on the present state of the Jewish poor in London - Book's seller - ",
 'Hoym, Karl Heinrich Graf von (Graf, Minister) - Apologie der Menschenrechte oder philosophisch kritische Beleuchtung der Schrift - Dedication - ',
 'Gregoire, Henri - Appel à la justice des nations et des rois - Personal Address - ',
 'Marcus, Benedikt - Bne-Zion - Approbation - Nikolsburg',
 'Stanislaus II. Poniatowski (König von Polen) - Die Juden Oder Die nothwendige Reformation der Juden in der Republik Polen - Dedication - ',
 'Hufnagel - Die Moseide in achtzehn Gesängen - Thanks - ',
 'Spalding - Die Moseide in achtzehn Gesängen - Thanks - ',
 'Ramler, Karl Wilhelm - Die Psalmen - Personal address - Berlin',
 'Joseph II. (Kaiser) - Einige jüdische Familienscenen, bey Erblickung des Patents über die Freyheiten, welche die Juden in den kaiserlichen Staaten erhalten haben - Dedication - Vienna',
 'Friedländer, Rebekka Meyer - Gebete der hochdeutschen u

In [11]:
productions = haskala_node[haskala_node['type'] == 'production']['title']
productions.to_list()

['Alexander, Isaak - אלכסנדר, יצחק - Abhandlung von der Freiheit des Menschen - Author',
 'Anonym - Actenstücke zur Geschichte der Erhebung der Juden zu Bürgern der Republik Batavien - Translator',
 "Friedländer, David - פרידלענדר, דוד - Akten-Stücke die Reform der Jüdischen Kolonieen in den Preußischen Staaten betreffend - Author,Mevi La'Dfus (Producer)",
 'אנונימי - An das Breslauer Publikum - Author',
 'Herz, Marcus - הרץ, מרכוס - An die Herausgeber des hebräischen Sammlers über die frühe Beerdigung der Juden - Author',
 'Maimon, Salomon - מימון, שלמה - Anfangsgründe der Newtonischen Philosophie - Author',
 'Alexander, Levy - אלכסנדר, לוי - Answer to Mr. Joshua Van Oven’s letters on the present state of the Jewish poor in London - Author',
 'Hirschel, Moses - הירשל, משה - Apologie der Menschenrechte oder philosophisch kritische Beleuchtung der Schrift - Author',
 'Hourwitz, Zalkind - הורביץ, זלקינד - Apologie des juifs en réponse à la question - Author',
 'Berr, Michel - בר, מיכאל -

In [12]:
books = haskala_node[haskala_node['type'] == 'book']['title']
books.to_list()
books.count()

np.int64(525)

In [13]:
editions = haskala_node[haskala_node['type'] == 'edition']['title']
editions.to_list()
editions.count()

np.int64(327)

In [14]:
prefaces = haskala_node[haskala_node['type'] == 'preface']['title']
prefaces.to_list()
prefaces.count()

np.int64(272)

In [15]:
translations = haskala_node[haskala_node['type'] == 'translation']['title']
translations.to_list()
translations.count()

np.int64(19)

In [16]:
haskala_field_data_field_full_title = pd.read_sql('SELECT * FROM field_data_field_full_title', con=db_connection)
haskala_field_data_field_full_title.head(5)

,entity_type,bundle,deleted,entity_id,revision_id,language,delta,field_full_title_value,field_full_title_format
0,node,book,0,1952,1952,und,0,Abhandlung von der Freiheit des Menschen,None
1,node,book,0,1953,1953,und,0,Actenstücke zur Geschichte der Erhebung der Ju...,None
2,node,book,0,1954,1954,und,0,Akten-Stücke die Reform der Jüdischen Koloniee...,None
3,node,book,0,1955,1955,und,0,An das Breslauer Publikum: ein Wort des Friede...,None
4,node,book,0,1956,1956,und,0,An die Herausgeber des hebräischen Sammlers üb...,None


In [17]:
haskala_field_data_field_full_title[haskala_field_data_field_full_title['field_full_title_value'].str.contains('Dresden', regex=False)]

,entity_type,bundle,deleted,entity_id,revision_id,language,delta,field_full_title_value,field_full_title_format
485,node,book,0,2448,2448,und,0,שיר תודה\n\nDanklied der Judenschaft zu Dresde...,None


In [18]:
haskala_field_data_field_full_title[haskala_field_data_field_full_title['field_full_title_value'].str.contains('Dresden', regex=False)]['entity_id']

485    2448
Name: entity_id, dtype: int64

In [19]:
haskala_field_revision_field_full_title = pd.read_sql('SELECT * FROM field_revision_field_full_title', con=db_connection)
haskala_field_revision_field_full_title.head(5)

,entity_type,bundle,deleted,entity_id,revision_id,language,delta,field_full_title_value,field_full_title_format
0,node,book,0,1952,1952,und,0,Abhandlung von der Freiheit des Menschen,None
1,node,book,0,1953,1953,und,0,Actenstücke zur Geschichte der Erhebung der Ju...,None
2,node,book,0,1954,1954,und,0,Akten-Stücke die Reform der Jüdischen Koloniee...,None
3,node,book,0,1955,1955,und,0,An das Breslauer Publikum: ein Wort des Friede...,None
4,node,book,0,1956,1956,und,0,An die Herausgeber des hebräischen Sammlers üb...,None


In [20]:
haskala_node_revision = pd.read_sql('SELECT * FROM node_revision', con=db_connection)
haskala_node_revision.head(5)

,nid,vid,uid,title,log,timestamp,status,comment,promote,sticky
0,1,1,1,Introduction,,1428490114,1,0,0,0
1,2,2,1,(Dr. med.) איסטרייכער,,1428490114,1,0,1,0
2,3,3,1,(Dr. med.) ווארבורג,,1428490114,1,0,1,0
3,4,4,1,(Dr.) בריינערסדארף,,1428490114,1,0,1,0
4,5,5,1,"(Feder)-Guttmann, Tuvia - פדר, טוביה",,1428490114,1,0,1,0


# List all  data fields

In [21]:
import pandas as pd

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'haskala'
  AND table_name LIKE 'field_data_field_%%'
ORDER BY table_name;
"""

field_tables = pd.read_sql(query, con=db_connection)
table_list= field_tables['TABLE_NAME'].to_list()
table_list

['field_data_field_alignment',
 'field_data_field_bans',
 'field_data_field_bar_ilan_library_id',
 'field_data_field_berlin_library_id',
 'field_data_field_bibliographical_citations',
 'field_data_field_book',
 'field_data_field_book_availability_notes',
 'field_data_field_book_not_available',
 'field_data_field_book_structure_notes',
 'field_data_field_book_studies',
 'field_data_field_book_type_general_notes',
 'field_data_field_british_library_id',
 'field_data_field_catalog_numbers_notes',
 'field_data_field_censorship',
 'field_data_field_contacts_official_agents',
 'field_data_field_contacts_other_people',
 'field_data_field_contemporary_disputes',
 'field_data_field_contemporary_references',
 'field_data_field_content',
 'field_data_field_contents_table_notes',
 'field_data_field_contradict_new_edition',
 'field_data_field_contradict_original',
 'field_data_field_copy_of_book_used',
 'field_data_field_date_of_birth',
 'field_data_field_date_of_death',
 'field_data_field_dedicati

In [22]:
# clean all field names
clean_names = [
    name[len("field_data_field_"):]
    for name in table_list
    if name.startswith("field_data_field_")
]
clean_names



['alignment',
 'bans',
 'bar_ilan_library_id',
 'berlin_library_id',
 'bibliographical_citations',
 'book',
 'book_availability_notes',
 'book_not_available',
 'book_structure_notes',
 'book_studies',
 'book_type_general_notes',
 'british_library_id',
 'catalog_numbers_notes',
 'censorship',
 'contacts_official_agents',
 'contacts_other_people',
 'contemporary_disputes',
 'contemporary_references',
 'content',
 'contents_table_notes',
 'contradict_new_edition',
 'contradict_original',
 'copy_of_book_used',
 'date_of_birth',
 'date_of_death',
 'dedications',
 'dedications_notes',
 'diagrams_book_pages',
 'diagrams_notes',
 'edition_changes',
 'edition_city',
 'edition_references',
 'edition_year',
 'editions_notes',
 'epilogue',
 'epilogue_notes',
 'examined_volume_number',
 'expanded_in_edition',
 'expanded_in_translations',
 'fonts',
 'format_of_publication_date',
 'founders_notes',
 'frankfurt_library_id',
 'full_title',
 'funders',
 'gender',
 'geolocation',
 'german_name',
 'gregor

In [23]:
# Generate mapping
field_map = {
    name: name.replace("field_data_field_", "")
    for name in table_list
}

field_map


{'field_data_field_alignment': 'alignment',
 'field_data_field_bans': 'bans',
 'field_data_field_bar_ilan_library_id': 'bar_ilan_library_id',
 'field_data_field_berlin_library_id': 'berlin_library_id',
 'field_data_field_bibliographical_citations': 'bibliographical_citations',
 'field_data_field_book': 'book',
 'field_data_field_book_availability_notes': 'book_availability_notes',
 'field_data_field_book_not_available': 'book_not_available',
 'field_data_field_book_structure_notes': 'book_structure_notes',
 'field_data_field_book_studies': 'book_studies',
 'field_data_field_book_type_general_notes': 'book_type_general_notes',
 'field_data_field_british_library_id': 'british_library_id',
 'field_data_field_catalog_numbers_notes': 'catalog_numbers_notes',
 'field_data_field_censorship': 'censorship',
 'field_data_field_contacts_official_agents': 'contacts_official_agents',
 'field_data_field_contacts_other_people': 'contacts_other_people',
 'field_data_field_contemporary_disputes': 'cont

In [24]:
query_rev = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'haskala'
  AND table_name LIKE 'field_revision_field_%%'
ORDER BY table_name;
"""

revision_tables = pd.read_sql(query_rev, con=db_connection)
revision_tables['TABLE_NAME'].tolist()


['field_revision_field_alignment',
 'field_revision_field_bans',
 'field_revision_field_bar_ilan_library_id',
 'field_revision_field_berlin_library_id',
 'field_revision_field_bibliographical_citations',
 'field_revision_field_book',
 'field_revision_field_book_availability_notes',
 'field_revision_field_book_not_available',
 'field_revision_field_book_structure_notes',
 'field_revision_field_book_studies',
 'field_revision_field_book_type_general_notes',
 'field_revision_field_british_library_id',
 'field_revision_field_catalog_numbers_notes',
 'field_revision_field_censorship',
 'field_revision_field_contacts_official_agents',
 'field_revision_field_contacts_other_people',
 'field_revision_field_contemporary_disputes',
 'field_revision_field_contemporary_references',
 'field_revision_field_content',
 'field_revision_field_contents_table_notes',
 'field_revision_field_contradict_new_edition',
 'field_revision_field_contradict_original',
 'field_revision_field_copy_of_book_used',
 'fie

## List all non-data fields

In [113]:
import pandas as pd

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'haskala'
  AND table_name NOT LIKE 'field_data_field_%%'
  AND table_name NOT LIKE 'field_revision_field_%%'
ORDER BY table_name;
"""

field_tables = pd.read_sql(query, con=db_connection)
table_list= field_tables['TABLE_NAME'].to_list()
table_list

['field_config',
 'field_config_instance',
 'menu_links',
 'menu_router',
 'migrate_group',
 'migrate_log',
 'migrate_map_haskalabooksmigrate',
 'migrate_map_haskalabooktranslationsmigrate',
 'migrate_map_haskalacitytermsmigrate',
 'migrate_map_haskalaeditionmigrate',
 'migrate_map_haskalaeditiontypetermsmigrate',
 'migrate_map_haskalamentionsmigrate',
 'migrate_map_haskalamodeltermsmigrate',
 'migrate_map_haskalaoriginaltypetermsmigrate',
 'migrate_map_haskalapageelementsmigrate',
 'migrate_map_haskalapeoplemigrate',
 'migrate_map_haskalaprefacesmigrate',
 'migrate_map_haskalaproductionsmigrate',
 'migrate_map_haskalareligiontermsmigrate',
 'migrate_status',
 'node',
 'node_access',
 'node_revision',
 'node_type',
 'registry',
 'registry_file',
 'role',
 'role_permission',
 'sequences',
 'system',
 'taxonomy_index',
 'taxonomy_term_data',
 'url_alias']

# Bundle Analyse

## Hilfsfunktionen

In [25]:
import re
import pandas as pd

# Erkennen, was für ein Feldtyp (Literal, Taxonomie, Node-Referenz, Datei, etc.) es ist
def infer_field_kind(column_name: str) -> tuple[str, str]:
    """
    Gibt (suffix, kind) zurück, z.B.
    ('value', 'literal'), ('tid', 'taxonomy'), ('target_id', 'node'), ('fid', 'file'), ...
    """
    if column_name.endswith('_value'):
        return 'value', 'literal'
    if column_name.endswith('_tid'):
        return 'tid', 'taxonomy'
    if column_name.endswith('_target_id'):
        return 'target_id', 'node'
    if column_name.endswith('_fid'):
        return 'fid', 'file'
    # Fallback
    return 'other', 'other'


def get_field_columns_for_table(table_name: str, limit: int = 50) -> pd.DataFrame:
    """
    Liest eine kleine Stichprobe aus einer field_data_field_* Tabelle ein
    und analysiert:
    - welche Bundles dort vorkommen
    - welche field_* Spalten es gibt
    """
    # Limit, damit es nicht explodiert
    df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT {limit}", con=db_connection)

    # alle Spalten, die mit 'field_' anfangen
    field_cols = [c for c in df.columns if c.startswith('field_')]

    records = []

    # welche Bundles kommen in dieser Tabelle überhaupt vor?
    bundles = df['bundle'].unique()

    for bundle in bundles:
        df_bundle = df[df['bundle'] == bundle]

        for col in field_cols:
            suffix, kind = infer_field_kind(col)

            # "Basisname" des Feldes, z.B. field_full_title_value -> field_full_title
            if suffix != 'other':
                base = re.sub(f'_{suffix}$', '', col)
            else:
                base = col

            records.append({
                'storage_table': table_name,
                'entity_type': df_bundle['entity_type'].iloc[0] if 'entity_type' in df_bundle.columns and len(df_bundle) else None,
                'bundle': bundle,
                'field_column': col,
                'field_base': base,
                'suffix': suffix,
                'field_kind': kind,
            })

    return pd.DataFrame(records)


## Taxonomien resolven

In [26]:
def enrich_node_relations(meta_df: pd.DataFrame) -> pd.DataFrame:
    """
    Für alle field_kind == 'node' (also *_target_id) wird ermittelt,
    auf welche Node-Typen (Bundles) verwiesen wird.
    """
    results = []

    for _, row in meta_df.iterrows():
        if row['field_kind'] != 'node':
            results.append(row)
            continue

        table = row['storage_table']
        col = row['field_column']

        query = f"""
        SELECT DISTINCT n.type
        FROM {table} AS f
        JOIN node AS n ON n.nid = f.{col}
        WHERE f.{col} IS NOT NULL
        LIMIT 10;
        """
        try:
            target_df = pd.read_sql(query, con=db_connection)
            target_types = sorted(target_df['type'].dropna().unique().tolist())
        except Exception as e:
            target_types = [f"ERROR: {e}"]

        row = row.copy()
        row['target_bundles'] = ", ".join(target_types) if target_types else None
        results.append(row)

    return pd.DataFrame(results)


def enrich_taxonomy_relations(meta_df: pd.DataFrame) -> pd.DataFrame:
    """
    Für alle field_kind == 'taxonomy' (also *_tid) wird ermittelt,
    aus welchen Vokabularen die TIDs stammen.
    """
    results = []

    for _, row in meta_df.iterrows():
        if row['field_kind'] != 'taxonomy':
            results.append(row)
            continue

        table = row['storage_table']
        col = row['field_column']

        query = f"""
        SELECT DISTINCT v.vid, v.name AS vocabulary
        FROM {table} AS f
        JOIN taxonomy_term_data AS t ON t.tid = f.{col}
        JOIN taxonomy_vocabulary AS v ON v.vid = t.vid
        WHERE f.{col} IS NOT NULL
        LIMIT 10;
        """
        try:
            vocab_df = pd.read_sql(query, con=db_connection)
            vocab_list = [
                f"{row2['vid']}:{row2['vocabulary']}"
                for _, row2 in vocab_df.iterrows()
            ]
        except Exception as e:
            vocab_list = [f"ERROR: {e}"]

        row = row.copy()
        row['vocabularies'] = ", ".join(sorted(set(vocab_list))) if vocab_list else None
        results.append(row)

    return pd.DataFrame(results)


In [27]:
all_records = []

for tbl in table_list:
    print("Analysiere Tabelle:", tbl)
    try:
        df_meta = get_field_columns_for_table(tbl)
        all_records.append(df_meta)
    except Exception as e:
        print("⚠️ Fehler bei", tbl, ":", e)

# Basis-Metadaten zu allen field_data_field_* Tabellen
bundle_field_meta = pd.concat(all_records, ignore_index=True)

# Node-Referenzen anreichern (target_bundles)
bundle_field_meta = enrich_node_relations(bundle_field_meta)

# Taxonomie-Referenzen anreichern (vocabularies)
bundle_field_meta = enrich_taxonomy_relations(bundle_field_meta)

bundle_field_meta.head()


Analysiere Tabelle: field_data_field_alignment
Analysiere Tabelle: field_data_field_bans
Analysiere Tabelle: field_data_field_bar_ilan_library_id
Analysiere Tabelle: field_data_field_berlin_library_id
Analysiere Tabelle: field_data_field_bibliographical_citations
Analysiere Tabelle: field_data_field_book
Analysiere Tabelle: field_data_field_book_availability_notes
Analysiere Tabelle: field_data_field_book_not_available
Analysiere Tabelle: field_data_field_book_structure_notes
Analysiere Tabelle: field_data_field_book_studies
Analysiere Tabelle: field_data_field_book_type_general_notes
Analysiere Tabelle: field_data_field_british_library_id
Analysiere Tabelle: field_data_field_catalog_numbers_notes
Analysiere Tabelle: field_data_field_censorship
Analysiere Tabelle: field_data_field_contacts_official_agents
Analysiere Tabelle: field_data_field_contacts_other_people
Analysiere Tabelle: field_data_field_contemporary_disputes
Analysiere Tabelle: field_data_field_contemporary_references
Anal

,storage_table,entity_type,bundle,field_column,field_base,suffix,field_kind,target_bundles,vocabularies
0,field_data_field_alignment,node,book,field_alignment_tid,field_alignment,tid,taxonomy,NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
1,field_data_field_bans,node,book,field_bans_value,field_bans,value,literal,NaN,NaN
2,field_data_field_bans,node,book,field_bans_format,field_bans_format,other,other,NaN,NaN
3,field_data_field_bar_ilan_library_id,node,book,field_bar_ilan_library_id_value,field_bar_ilan_library_id,value,literal,NaN,NaN
4,field_data_field_bar_ilan_library_id,node,book,field_bar_ilan_library_id_format,field_bar_ilan_library_id_format,other,other,NaN,NaN


## Gruppierungen

In [28]:
bundle_field_meta[bundle_field_meta['bundle'] == 'book'] \
    .sort_values(['field_base', 'suffix']) \
    .reset_index(drop=True)


,storage_table,entity_type,bundle,field_column,field_base,suffix,field_kind,target_bundles,vocabularies
0,field_data_field_alignment,node,book,field_alignment_tid,field_alignment,tid,taxonomy,NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
1,field_data_field_bans,node,book,field_bans_value,field_bans,value,literal,NaN,NaN
2,field_data_field_bans,node,book,field_bans_format,field_bans_format,other,other,NaN,NaN
3,field_data_field_bar_ilan_library_id,node,book,field_bar_ilan_library_id_value,field_bar_ilan_library_id,value,literal,NaN,NaN
4,field_data_field_bar_ilan_library_id,node,book,field_bar_ilan_library_id_format,field_bar_ilan_library_id_format,other,other,NaN,NaN
...,...,...,...,...,...,...,...,...,...
261,field_data_field_volumes_notes,node,book,field_volumes_notes_format,field_volumes_notes_format,other,other,NaN,NaN
262,field_data_field_volumes_published_number,node,book,field_volumes_published_number_value,field_volumes_published_number,value,literal,NaN,NaN
263,field_data_field_volumes_published_number,node,book,field_volumes_published_number_format,field_volumes_published_number_format,other,other,NaN,NaN
264,field_data_field_width,node,book,field_width_value,field_width,value,literal,NaN,NaN


## Alles kombinieren

In [29]:
bundle_field_meta[
    (bundle_field_meta['field_kind'] == 'node') &
    (bundle_field_meta['target_bundles'].str.contains('book', na=False))
][['bundle', 'field_base', 'storage_table', 'field_column', 'target_bundles']]


,bundle,field_base,storage_table,field_column,target_bundles
9,translation,field_book,field_data_field_book,field_book_target_id,book
10,edition,field_book,field_data_field_book,field_book_target_id,book


In [30]:
bundle_field_meta[
    (bundle_field_meta['field_kind'] == 'node') &
    (bundle_field_meta['target_bundles'].str.contains('person', na=False))
][['bundle', 'field_base', 'storage_table', 'field_column', 'target_bundles']]


,bundle,field_base,storage_table,field_column,target_bundles
117,mention,field_mentionee,field_data_field_mentionee,field_mentionee_target_id,person
151,book,field_old_text_author,field_data_field_old_text_author,field_old_text_author_target_id,person
169,book,field_original_text_author,field_data_field_original_text_author,field_original_text_author_target_id,person
206,preface,field_preface_writer,field_data_field_preface_writer,field_preface_writer_target_id,person
234,production,field_producer,field_data_field_producer,field_producer_target_id,person
313,translation,field_translator,field_data_field_translator,field_translator_target_id,person


In [31]:
bundle_field_meta[
    (bundle_field_meta['bundle'] == 'person') &
    (bundle_field_meta['field_kind'] == 'taxonomy')
][['field_base', 'field_column', 'vocabularies']]


,field_base,field_column,vocabularies
77,field_gender,field_gender_tid,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
144,field_occupation,field_occupation_tid,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
195,field_place_of_birth,field_place_of_birth_tid,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
196,field_place_of_death,field_place_of_death_tid,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."


In [32]:
import pandas as pd
import re

# 1. Filter nur auf die Bundles 'book' und 'person'
subset = bundle_field_meta[
    bundle_field_meta['bundle'].isin(['book', 'person'])
].copy()

# 2. Hilfsfunktion: SQL-Typ aus DESCRIBE holen und grob klassifizieren
def get_table_column_types(table_name: str):
    """
    Gibt ein Dict {column_name: sql_type_string} zurück, z.B. {'field_full_title_value': 'text', ...}
    """
    df = pd.read_sql(f"DESCRIBE {table_name}", con=db_connection)
    type_map = {}
    for _, row in df.iterrows():
        col = row['Field']
        t = row['Type']  # z.B. 'varchar(255)', 'int(11)', 'text'
        # Nur den Basis-Typ ohne Länge extrahieren
        m = re.match(r'([a-zA-Z]+)', t)
        base = m.group(1).lower() if m else t
        type_map[col] = base
    return type_map

# 3. Alle beteiligten Tabellen sammeln und Typ-Maps bauen
table_type_maps = {}
for tbl in subset['storage_table'].unique():
    try:
        table_type_maps[tbl] = get_table_column_types(tbl)
    except Exception as e:
        print("⚠️ Konnte DESCRIBE nicht lesen für", tbl, ":", e)
        table_type_maps[tbl] = {}

# 4. SQL-Typ in das DataFrame einfügen
sql_types = []
for _, row in subset.iterrows():
    tbl = row['storage_table']
    col = row['field_column']
    sql_type = table_type_maps.get(tbl, {}).get(col, None)
    sql_types.append(sql_type)

subset['sql_type'] = sql_types

# 5. Heuristik für Django-Feldtyp
def suggest_django_field(row):
    kind = row['field_kind']
    sql_type = (row['sql_type'] or '').lower()
    col = row['field_column']

    # Node-Referenzen (z.B. *_target_id)
    if kind == 'node':
        return "models.IntegerField  # nid (FK möglich)"

    # Taxonomy-Referenzen (z.B. *_tid)
    if kind == 'taxonomy':
        return "models.IntegerField  # tid (FK zu Vokabular möglich)"

    # Dateien
    if kind == 'file':
        return "models.IntegerField  # fid (FileRef, später FileField?)"

    # Literale
    if kind == 'literal':
        # grobe Zuordnung nach SQL-Typ
        if sql_type in ('text', 'mediumtext', 'longtext'):
            return "models.TextField"
        if sql_type in ('varchar', 'char', 'tinytext'):
            return "models.CharField(max_length=255)"
        if sql_type in ('int', 'bigint', 'smallint', 'tinyint'):
            return "models.IntegerField"
        if sql_type in ('decimal', 'numeric', 'float', 'double'):
            return "models.FloatField"
        if sql_type in ('date', 'datetime', 'timestamp'):
            return "models.CharField(max_length=50)  # oder DateField/DateTimeField"
        # Fallback
        return "models.TextField"

    # alles andere
    return "models.TextField  # (prüfen)"

subset['django_field_suggestion'] = subset.apply(suggest_django_field, axis=1)

# 6. Deduplizieren: eine Zeile pro (bundle, field_base, suffix)
compact = (
    subset
    .sort_values(['bundle', 'field_base', 'suffix'])
    .drop_duplicates(subset=['bundle', 'field_base', 'suffix'])
    [[
        'bundle',
        'field_base',
        'suffix',
        'field_kind',
        'storage_table',
        'field_column',
        'sql_type',
        'django_field_suggestion',
        'target_bundles',
        'vocabularies',
    ]]
    .reset_index(drop=True)
)

compact


,bundle,field_base,suffix,field_kind,storage_table,field_column,sql_type,django_field_suggestion,target_bundles,vocabularies
0,book,field_alignment,tid,taxonomy,field_data_field_alignment,field_alignment_tid,int,models.IntegerField # tid (FK zu Vokabular mö...,NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
1,book,field_bans,value,literal,field_data_field_bans,field_bans_value,longtext,models.TextField,NaN,NaN
2,book,field_bans_format,other,other,field_data_field_bans,field_bans_format,varchar,models.TextField # (prüfen),NaN,NaN
3,book,field_bar_ilan_library_id,value,literal,field_data_field_bar_ilan_library_id,field_bar_ilan_library_id_value,varchar,models.CharField(max_length=255),NaN,NaN
4,book,field_bar_ilan_library_id_format,other,other,field_data_field_bar_ilan_library_id,field_bar_ilan_library_id_format,varchar,models.TextField # (prüfen),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
277,person,field_place_of_death,tid,taxonomy,field_data_field_place_of_death,field_place_of_death_tid,int,models.IntegerField # tid (FK zu Vokabular mö...,NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
278,person,field_pseudonym,value,literal,field_data_field_pseudonym,field_pseudonym_value,varchar,models.CharField(max_length=255),NaN,NaN
279,person,field_pseudonym_format,other,other,field_data_field_pseudonym,field_pseudonym_format,varchar,models.TextField # (prüfen),NaN,NaN
280,person,field_viaf_id,value,literal,field_data_field_viaf_id,field_viaf_id_value,varchar,models.CharField(max_length=255),NaN,NaN


In [33]:
import pandas as pd
import re

# 1. Nur Bundle 'person' herausfiltern
persons_meta = bundle_field_meta[bundle_field_meta['bundle'] == 'person'].copy()

# 2. SQL-Typen der Spalten je Tabelle holen
def get_table_column_types(table_name: str):
    """
    Gibt ein Dict {column_name: sql_type_string} zurück, z.B. {'field_german_name_value': 'text', ...}
    """
    df = pd.read_sql(f"DESCRIBE {table_name}", con=db_connection)
    type_map = {}
    for _, row in df.iterrows():
        col = row['Field']
        t = row['Type']  # z.B. 'varchar(255)', 'int(11)', 'text'
        m = re.match(r'([a-zA-Z]+)', t)
        base = m.group(1).lower() if m else t
        type_map[col] = base
    return type_map

# 3. Typ-Maps bauen
table_type_maps = {}
for tbl in persons_meta['storage_table'].unique():
    try:
        table_type_maps[tbl] = get_table_column_types(tbl)
    except Exception as e:
        print("⚠️ Konnte DESCRIBE nicht lesen für", tbl, ":", e)
        table_type_maps[tbl] = {}

# 4. SQL-Typ-Spalte ergänzen
sql_types = []
for _, row in persons_meta.iterrows():
    tbl = row['storage_table']
    col = row['field_column']
    sql_type = table_type_maps.get(tbl, {}).get(col, None)
    sql_types.append(sql_type)

persons_meta['sql_type'] = sql_types

# 5. Heuristik: Django-Feld vorschlagen
def suggest_django_field(row):
    kind = row['field_kind']
    sql_type = (row['sql_type'] or '').lower()

    if kind == 'node':
        return "models.IntegerField  # nid (FK zu anderem Node möglich)"

    if kind == 'taxonomy':
        return "models.IntegerField  # tid (FK zu Taxonomie, z.B. Gender, Occupation)"

    if kind == 'file':
        return "models.IntegerField  # fid (FileRef)"

    if kind == 'literal':
        if sql_type in ('text', 'mediumtext', 'longtext'):
            return "models.TextField"
        if sql_type in ('varchar', 'char', 'tinytext'):
            return "models.CharField(max_length=255)"
        if sql_type in ('int', 'bigint', 'smallint', 'tinyint'):
            return "models.IntegerField"
        if sql_type in ('decimal', 'numeric', 'float', 'double'):
            return "models.FloatField"
        if sql_type in ('date', 'datetime', 'timestamp'):
            return "models.CharField(max_length=50)  # oder DateField/DateTimeField"
        return "models.TextField"

    return "models.TextField  # (prüfen)"

persons_meta['django_field_suggestion'] = persons_meta.apply(suggest_django_field, axis=1)

# 6. Kompakte Tabelle: eine Zeile pro (field_base, suffix)
compact_person = (
    persons_meta
    .sort_values(['field_base', 'suffix'])
    .drop_duplicates(subset=['field_base', 'suffix'])
    [[
        'bundle',
        'field_base',
        'suffix',
        'field_kind',
        'storage_table',
        'field_column',
        'sql_type',
        'django_field_suggestion',
        'target_bundles',
        'vocabularies',
    ]]
    .reset_index(drop=True)
)

compact_person


,bundle,field_base,suffix,field_kind,storage_table,field_column,sql_type,django_field_suggestion,target_bundles,vocabularies
0,person,field_date_of_birth,value,literal,field_data_field_date_of_birth,field_date_of_birth_value,varchar,models.CharField(max_length=255),NaN,NaN
1,person,field_date_of_birth_format,other,other,field_data_field_date_of_birth,field_date_of_birth_format,varchar,models.TextField # (prüfen),NaN,NaN
2,person,field_date_of_death,value,literal,field_data_field_date_of_death,field_date_of_death_value,varchar,models.CharField(max_length=255),NaN,NaN
3,person,field_date_of_death_format,other,other,field_data_field_date_of_death,field_date_of_death_format,varchar,models.TextField # (prüfen),NaN,NaN
4,person,field_gender,tid,taxonomy,field_data_field_gender,field_gender_tid,int,"models.IntegerField # tid (FK zu Taxonomie, z...",NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."
5,person,field_german_name,value,literal,field_data_field_german_name,field_german_name_value,varchar,models.CharField(max_length=255),NaN,NaN
6,person,field_german_name_format,other,other,field_data_field_german_name,field_german_name_format,varchar,models.TextField # (prüfen),NaN,NaN
7,person,field_hebrew_name,value,literal,field_data_field_hebrew_name,field_hebrew_name_value,varchar,models.CharField(max_length=255),NaN,NaN
8,person,field_hebrew_name_format,other,other,field_data_field_hebrew_name,field_hebrew_name_format,varchar,models.TextField # (prüfen),NaN,NaN
9,person,field_occupation,tid,taxonomy,field_data_field_occupation,field_occupation_tid,int,"models.IntegerField # tid (FK zu Taxonomie, z...",NaN,"ERROR: (pymysql.err.ProgrammingError) (1146, ""..."


## Cities

In [34]:
bundle_field_meta[bundle_field_meta['entity_type'] == 'taxonomy_term']['bundle'].unique()


array(['cities'], dtype=object)

In [35]:
import pandas as pd
import re

# 1. Nur Taxonomie-Terme, Bundle 'cities' (Places)
places_meta = bundle_field_meta[
    (bundle_field_meta['entity_type'] == 'taxonomy_term') &
    (bundle_field_meta['bundle'] == 'cities')
].copy()

# 2. SQL-Typen je Tabelle holen
def get_table_column_types(table_name: str):
    """
    Gibt ein Dict {column_name: sql_type_string} zurück,
    z.B. {'field_geolocation_lat': 'double', ...}
    """
    df = pd.read_sql(f"DESCRIBE {table_name}", con=db_connection)
    type_map = {}
    for _, row in df.iterrows():
        col = row['Field']
        t = row['Type']  # z.B. 'varchar(255)', 'int(11)', 'text', 'double'
        m = re.match(r'([a-zA-Z]+)', t)
        base = m.group(1).lower() if m else t
        type_map[col] = base
    return type_map

# 3. Typ-Maps bauen
table_type_maps = {}
for tbl in places_meta['storage_table'].unique():
    try:
        table_type_maps[tbl] = get_table_column_types(tbl)
    except Exception as e:
        print("⚠️ Konnte DESCRIBE nicht lesen für", tbl, ":", e)
        table_type_maps[tbl] = {}

# 4. SQL-Typ-Spalte ergänzen
sql_types = []
for _, row in places_meta.iterrows():
    tbl = row['storage_table']
    col = row['field_column']
    sql_type = table_type_maps.get(tbl, {}).get(col, None)
    sql_types.append(sql_type)

places_meta['sql_type'] = sql_types

# 5. Heuristik: Django-Feld vorschlagen
def suggest_django_field(row):
    kind = row['field_kind']
    sql_type = (row['sql_type'] or '').lower()

    if kind == 'node':
        return "models.IntegerField  # nid (FK zu Node möglich)"

    if kind == 'taxonomy':
        return "models.IntegerField  # tid (FK zu anderer Taxonomie)"

    if kind == 'file':
        return "models.IntegerField  # fid (FileRef)"

    if kind == 'literal':
        if sql_type in ('text', 'mediumtext', 'longtext'):
            return "models.TextField"
        if sql_type in ('varchar', 'char', 'tinytext'):
            return "models.CharField(max_length=255)"
        if sql_type in ('int', 'bigint', 'smallint', 'tinyint'):
            return "models.IntegerField"
        if sql_type in ('decimal', 'numeric', 'float', 'double', 'doubleprecision'):
            return "models.FloatField"
        if sql_type in ('date', 'datetime', 'timestamp'):
            return "models.CharField(max_length=50)  # oder DateField/DateTimeField"
        return "models.TextField"

    return "models.TextField  # (prüfen)"

places_meta['django_field_suggestion'] = places_meta.apply(suggest_django_field, axis=1)

# 6. Kompakte Tabelle: eine Zeile pro (field_base, suffix)
compact_places = (
    places_meta
    .sort_values(['field_base', 'suffix'])
    .drop_duplicates(subset=['field_base', 'suffix'])
    [[
        'bundle',
        'field_base',
        'suffix',
        'field_kind',
        'storage_table',
        'field_column',
        'sql_type',
        'django_field_suggestion',
        'target_bundles',
        'vocabularies',
    ]]
    .reset_index(drop=True)
)

compact_places


,bundle,field_base,suffix,field_kind,storage_table,field_column,sql_type,django_field_suggestion,target_bundles,vocabularies
0,cities,field_geolocation_lat,other,other,field_data_field_geolocation,field_geolocation_lat,double,models.TextField # (prüfen),NaN,NaN
1,cities,field_geolocation_lat_cos,other,other,field_data_field_geolocation,field_geolocation_lat_cos,double,models.TextField # (prüfen),NaN,NaN
2,cities,field_geolocation_lat_sin,other,other,field_data_field_geolocation,field_geolocation_lat_sin,double,models.TextField # (prüfen),NaN,NaN
3,cities,field_geolocation_lng,other,other,field_data_field_geolocation,field_geolocation_lng,double,models.TextField # (prüfen),NaN,NaN
4,cities,field_geolocation_lng_rad,other,other,field_data_field_geolocation,field_geolocation_lng_rad,double,models.TextField # (prüfen),NaN,NaN


## Heuristik für Geolocation-Felder korrigieren

In [36]:
# Alle double-Spalten als FloatField behandeln
mask_double = compact_places['sql_type'].str.lower() == 'double'
compact_places.loc[mask_double, 'django_field_suggestion'] = "models.FloatField()"

compact_places


,bundle,field_base,suffix,field_kind,storage_table,field_column,sql_type,django_field_suggestion,target_bundles,vocabularies
0,cities,field_geolocation_lat,other,other,field_data_field_geolocation,field_geolocation_lat,double,models.FloatField(),NaN,NaN
1,cities,field_geolocation_lat_cos,other,other,field_data_field_geolocation,field_geolocation_lat_cos,double,models.FloatField(),NaN,NaN
2,cities,field_geolocation_lat_sin,other,other,field_data_field_geolocation,field_geolocation_lat_sin,double,models.FloatField(),NaN,NaN
3,cities,field_geolocation_lng,other,other,field_data_field_geolocation,field_geolocation_lng,double,models.FloatField(),NaN,NaN
4,cities,field_geolocation_lng_rad,other,other,field_data_field_geolocation,field_geolocation_lng_rad,double,models.FloatField(),NaN,NaN


In [37]:
def suggest_django_field(row):
    kind = row['field_kind']
    sql_type = (row['sql_type'] or '').lower()

    # Node-Referenzen (z.B. *_target_id)
    if kind == 'node':
        return "models.IntegerField  # nid (FK möglich)"

    # Taxonomy-Referenzen (z.B. *_tid)
    if kind == 'taxonomy':
        return "models.IntegerField  # tid (FK zu Vokabular möglich)"

    # Dateien
    if kind == 'file':
        return "models.IntegerField  # fid (FileRef)"

    # Für alle anderen Fälle schauen wir direkt auf den SQL-Typ
    if sql_type in ('double', 'float', 'decimal', 'numeric', 'doubleprecision'):
        return "models.FloatField()"
    if sql_type in ('int', 'bigint', 'smallint', 'tinyint'):
        return "models.IntegerField()"
    if sql_type in ('text', 'mediumtext', 'longtext'):
        return "models.TextField()"
    if sql_type in ('varchar', 'char', 'tinytext'):
        return "models.CharField(max_length=255)"
    if sql_type in ('date', 'datetime', 'timestamp'):
        return "models.CharField(max_length=50)  # oder DateField/DateTimeField"

    return "models.TextField()  # (prüfen)"


# Export

In [38]:
from sqlalchemy import create_engine
engine = create_engine( f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
db_connection = engine.connect()


In [39]:
# taxonomy_term_data für Cities
cities_sql = """
SELECT t.tid, t.vid, t.name, t.description
FROM taxonomy_term_data AS t
WHERE t.vid = 1  -- 1 = Cities
"""
cities_df = pd.read_sql(cities_sql, con=db_connection)

# field_data_field_geolocation
geo_sql = """
SELECT entity_id AS tid,
       field_geolocation_lat,
       field_geolocation_lng,
       field_geolocation_lat_sin,
       field_geolocation_lat_cos,
       field_geolocation_lng_rad
FROM field_data_field_geolocation
WHERE entity_type = 'taxonomy_term'
  AND bundle = 'cities'
"""
geo_df = pd.read_sql(geo_sql, con=db_connection)

cities_geo = cities_df.merge(geo_df, on='tid', how='left')

cities_geo.to_csv("export/cities_with_geolocation.csv", index=False)
cities_geo.head()


,tid,vid,name,description,field_geolocation_lat,field_geolocation_lng,field_geolocation_lat_sin,field_geolocation_lat_cos,field_geolocation_lng_rad
0,1,1,Altona,None,53.55000,9.93333,0.804376,0.594121,0.173369
1,2,1,Altstadt,None,53.62805,11.41582,0.805184,0.593025,0.199244
2,3,1,Amsterdam,None,52.37403,4.88969,0.792013,0.610504,0.085341
3,4,1,Anklam,None,53.85637,13.68965,0.807541,0.589811,0.238929
4,5,1,Anspach,None,50.30107,8.51020,0.769411,0.638753,0.148531


In [40]:
cities_sql = """
SELECT t.tid, t.vid, t.name, t.description
FROM taxonomy_term_data AS t
WHERE t.vid = 1
"""
cities_df = pd.read_sql(cities_sql, con=db_connection)

geo_sql = """
SELECT entity_id AS tid,
       field_geolocation_lat,
       field_geolocation_lng,
       field_geolocation_lat_sin,
       field_geolocation_lat_cos,
       field_geolocation_lng_rad
FROM field_data_field_geolocation
WHERE entity_type = 'taxonomy_term'
  AND bundle = 'cities'
"""
geo_df = pd.read_sql(geo_sql, con=db_connection)


In [41]:
cities_df = pd.read_sql(cities_sql, con=engine)
geo_df    = pd.read_sql(geo_sql,    con=engine)


# Persons export

In [42]:
import pandas as pd
import re

# 1. Basis: alle Nodes vom Typ 'person'
persons = pd.read_sql(
    """
    SELECT nid, vid, type, title, status, created, changed, uid
    FROM node
    WHERE type = 'person'
    """,
    con=db_connection,
)

# 2. Metadaten nur für bundle 'person'
persons_meta = bundle_field_meta[bundle_field_meta['bundle'] == 'person'].copy()

# Nur echte Feld-Basen (ohne Duplikate)
field_bases = sorted(persons_meta['field_base'].unique())

print("Felder für bundle 'person':", len(field_bases))

for field_base in field_bases:
    # z.B. field_base = 'field_german_name'
    subset = persons_meta[persons_meta['field_base'] == field_base]

    # welche Tabelle?
    table_name = subset['storage_table'].iloc[0]
    field_columns = subset['field_column'].unique().tolist()

    print(f"  -> Merge {field_base} aus {table_name} ({', '.join(field_columns)})")

    # 3. Feld-Tabelle lesen
    # Wir nehmen:
    # - entity_type = 'node'
    # - bundle = 'person'
    # - deleted = 0
    # - delta = 0 (erste Ausprägung)
    # - Sprache: bevorzugt 'und' / 'de' / 'en'
    sql = f"""
    SELECT entity_id AS nid, {", ".join(field_columns)}
    FROM {table_name}
    WHERE entity_type = 'node'
      AND bundle = 'person'
      AND deleted = 0
      AND delta = 0
    """
    field_df = pd.read_sql(sql, con=db_connection)

    # Merge auf Basis 'nid'
    persons = persons.merge(field_df, on='nid', how='left')

print("Gesamte Personentabelle hat Spalten:", len(persons.columns))

# 4. Spalten sinnvoll umbenennen (optional)
# z.B. field_german_name_value -> german_name
rename_map = {}
for col in persons.columns:
    if col.startswith("field_"):
        new = col
        # field_ vorweg weg
        new = re.sub(r"^field_", "", new)
        # typische Suffixe wegnehmen
        new = re.sub(r"_value$", "", new)
        # *_tid, *_target_id, *_fid lassen wir erkennbar
        rename_map[col] = new

persons = persons.rename(columns=rename_map)

# 5. Exportieren
persons.to_csv("export/persons_for_django.csv", index=False)
persons.head()


Felder für bundle 'person': 16
  -> Merge field_date_of_birth aus field_data_field_date_of_birth (field_date_of_birth_value)
  -> Merge field_date_of_birth_format aus field_data_field_date_of_birth (field_date_of_birth_format)
  -> Merge field_date_of_death aus field_data_field_date_of_death (field_date_of_death_value)
  -> Merge field_date_of_death_format aus field_data_field_date_of_death (field_date_of_death_format)
  -> Merge field_gender aus field_data_field_gender (field_gender_tid)
  -> Merge field_german_name aus field_data_field_german_name (field_german_name_value)
  -> Merge field_german_name_format aus field_data_field_german_name (field_german_name_format)
  -> Merge field_hebrew_name aus field_data_field_hebrew_name (field_hebrew_name_value)
  -> Merge field_hebrew_name_format aus field_data_field_hebrew_name (field_hebrew_name_format)
  -> Merge field_occupation aus field_data_field_occupation (field_occupation_tid)
  -> Merge field_place_of_birth aus field_data_field_pl

,nid,vid,type,title,status,created,changed,uid,date_of_birth,date_of_birth_format,...,german_name_format,hebrew_name,hebrew_name_format,occupation_tid,place_of_birth_tid,place_of_death_tid,pseudonym,pseudonym_format,viaf_id,viaf_id_format
0,2,2,person,(Dr. med.) איסטרייכער,1,1428490114,1428490114,1,NaN,NaN,...,None,איסטרייכער,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,3,person,(Dr. med.) ווארבורג,1,1428490114,1428490114,1,NaN,NaN,...,None,ווארבורג,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,4,person,(Dr.) בריינערסדארף,1,1428490114,1428490114,1,NaN,NaN,...,None,בריינערסדארף,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,5,person,"(Feder)-Guttmann, Tuvia - פדר, טוביה",1,1428490114,1428490114,1,NaN,NaN,...,NaN,NaN,NaN,402.0,NaN,NaN,NaN,NaN,NaN,NaN
4,6,6,person,"Aaron, Joseph Philipp - אהרן, יוסף",1,1428490114,1428490114,1,NaN,NaN,...,NaN,NaN,NaN,402.0,NaN,NaN,NaN,NaN,297651133,None


# Books export

In [43]:
import pandas as pd
import re

# 1. Basis: alle Nodes vom Typ 'book'
books = pd.read_sql(
    """
    SELECT nid, vid, type, title, status, created, changed, uid
    FROM node
    WHERE type = 'book'
    """,
    con=db_connection,
)

# 2. Metadaten nur für bundle 'book'
books_meta = bundle_field_meta[bundle_field_meta['bundle'] == 'book'].copy()

field_bases_book = sorted(books_meta['field_base'].unique())

print("Felder für bundle 'book':", len(field_bases_book))

for field_base in field_bases_book:
    subset = books_meta[books_meta['field_base'] == field_base]

    table_name = subset['storage_table'].iloc[0]
    field_columns = subset['field_column'].unique().tolist()

    print(f"  -> Merge {field_base} aus {table_name} ({', '.join(field_columns)})")

    sql = f"""
    SELECT entity_id AS nid, {", ".join(field_columns)}
    FROM {table_name}
    WHERE entity_type = 'node'
      AND bundle = 'book'
      AND deleted = 0
      AND delta = 0
    """
    field_df = pd.read_sql(sql, con=db_connection)

    books = books.merge(field_df, on='nid', how='left')

print("Gesamte Büchertabelle hat Spalten:", len(books.columns))

# 3. Spalten sinnvoll umbenennen
rename_map = {}
for col in books.columns:
    if col.startswith("field_"):
        new = col
        new = re.sub(r"^field_", "", new)
        new = re.sub(r"_value$", "", new)
        rename_map[col] = new

books = books.rename(columns=rename_map)

# 4. Exportieren
books.to_csv("export/books_for_django.csv", index=False)
books.head()


Felder für bundle 'book': 266
  -> Merge field_alignment aus field_data_field_alignment (field_alignment_tid)
  -> Merge field_bans aus field_data_field_bans (field_bans_value)
  -> Merge field_bans_format aus field_data_field_bans (field_bans_format)
  -> Merge field_bar_ilan_library_id aus field_data_field_bar_ilan_library_id (field_bar_ilan_library_id_value)
  -> Merge field_bar_ilan_library_id_format aus field_data_field_bar_ilan_library_id (field_bar_ilan_library_id_format)
  -> Merge field_berlin_library_id aus field_data_field_berlin_library_id (field_berlin_library_id_value)
  -> Merge field_berlin_library_id_format aus field_data_field_berlin_library_id (field_berlin_library_id_format)
  -> Merge field_bibliographical_citations aus field_data_field_bibliographical_citations (field_bibliographical_citations_value)
  -> Merge field_bibliographical_citations_format aus field_data_field_bibliographical_citations (field_bibliographical_citations_format)
  -> Merge field_book_availa

,nid,vid,type,title,status,created,changed,uid,alignment_tid,bans,...,translation_notes,translation_notes_format,translation_type,typography_tid,volumes_notes,volumes_notes_format,volumes_published_number,volumes_published_number_format,width,width_format
0,1952,1952,book,Abhandlung von der Freiheit des Menschen,1,1428490158,1428490158,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1953,1953,book,Actenstücke zur Geschichte der Erhebung der Ju...,1,1428490158,1428490158,1,412.0,NaN,...,NaN,NaN,NaN,423.0,NaN,NaN,NaN,NaN,10.1,None
2,1954,1954,book,Akten-Stücke die Reform der Jüdischen Koloniee...,1,1428490158,1428490158,1,412.0,NaN,...,NaN,NaN,NaN,429.0,NaN,NaN,1,None,11.3,None
3,1955,1955,book,An das Breslauer Publikum,1,1428490158,1428490158,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1956,1956,book,An die Herausgeber des hebräischen Sammlers üb...,1,1428490158,1428490158,1,NaN,NaN,...,NaN,NaN,NaN,429.0,NaN,NaN,NaN,NaN,NaN,NaN


# Hilfsmodelle exportieren

In [44]:
import pandas as pd
import re
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(exist_ok=True)

def export_bundle(bundle_name: str, out_filename: str):
    """
    Exportiert alle Nodes eines Bundles (edition, translation, preface, mention, production)
    inkl. aller field_data_field_* Felder in eine breite CSV.
    """
    print(f"=== Export für Bundle: {bundle_name} ===")

    # 1. Basis-Node-Tabelle
    nodes = pd.read_sql(
        f"""
        SELECT nid, vid, type, title, status, created, changed, uid
        FROM node
        WHERE type = '{bundle_name}'
        """,
        con=db_connection,
    )

    if nodes.empty:
        print(f"⚠️ Keine Nodes vom Typ '{bundle_name}' gefunden.")
        return

    print(f"  {len(nodes)} Nodes gefunden.")

    # 2. Felder-Metadaten für dieses Bundle
    meta = bundle_field_meta[bundle_field_meta['bundle'] == bundle_name].copy()
    if meta.empty:
        print(f"⚠️ Keine field_data_field_* Einträge für bundle='{bundle_name}' in bundle_field_meta.")
        return

    field_bases = sorted(meta['field_base'].unique())
    print(f"  {len(field_bases)} verschiedene Feldgruppen (field_base) gefunden.")

    df = nodes.copy()

    # 3. Pro Feldbasis (field_base) die entsprechende field_data_field_* Tabelle mergen
    for field_base in field_bases:
        subset = meta[meta['field_base'] == field_base]
        table_name = subset['storage_table'].iloc[0]
        field_columns = subset['field_column'].unique().tolist()

        print(f"    -> Merge {field_base} aus {table_name} ({', '.join(field_columns)})")

        sql = f"""
        SELECT entity_id AS nid, {", ".join(field_columns)}
        FROM {table_name}
        WHERE entity_type = 'node'
          AND bundle = '{bundle_name}'
          AND deleted = 0
          AND delta = 0
        """

        try:
            field_df = pd.read_sql(sql, con=db_connection)
        except Exception as e:
            print(f"      ⚠️ Fehler beim Lesen von {table_name}: {e}")
            continue

        df = df.merge(field_df, on='nid', how='left')

    print(f"  Ergebnis hat {df.shape[0]} Zeilen und {df.shape[1]} Spalten.")

    # 4. Spaltennamen etwas säubern
    rename_map = {}
    for col in df.columns:
        if col.startswith("field_"):
            new = col
            new = re.sub(r"^field_", "", new)
            new = re.sub(r"_value$", "", new)
            rename_map[col] = new

    df = df.rename(columns=rename_map)

    # 5. CSV speichern
    out_path = EXPORT_DIR / out_filename
    df.to_csv(out_path, index=False)
    print(f"✅ Export gespeichert unter: {out_path}\n")


In [45]:
export_bundle("edition",     "editions_for_django.csv")
export_bundle("translation", "translations_for_django.csv")
export_bundle("preface",     "prefaces_for_django.csv")
export_bundle("mention",     "mentions_for_django.csv")
export_bundle("production",  "productions_for_django.csv")


=== Export für Bundle: edition ===
  327 Nodes gefunden.
  8 verschiedene Feldgruppen (field_base) gefunden.
    -> Merge field_book aus field_data_field_book (field_book_target_id)
    -> Merge field_edition_changes aus field_data_field_edition_changes (field_edition_changes_value)
    -> Merge field_edition_changes_format aus field_data_field_edition_changes (field_edition_changes_format)
    -> Merge field_edition_city aus field_data_field_edition_city (field_edition_city_tid)
    -> Merge field_edition_references aus field_data_field_edition_references (field_edition_references_value)
    -> Merge field_edition_references_format aus field_data_field_edition_references (field_edition_references_format)
    -> Merge field_edition_year aus field_data_field_edition_year (field_edition_year_value)
    -> Merge field_edition_year_format aus field_data_field_edition_year (field_edition_year_format)
  Ergebnis hat 327 Zeilen und 16 Spalten.
✅ Export gespeichert unter: export/editions_for_d

# Checks for export

## Base check

In [46]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")

editions      = pd.read_csv(EXPORT_DIR / "editions_for_django.csv")
translations  = pd.read_csv(EXPORT_DIR / "translations_for_django.csv")
prefaces      = pd.read_csv(EXPORT_DIR / "prefaces_for_django.csv")
mentions      = pd.read_csv(EXPORT_DIR / "mentions_for_django.csv")
productions   = pd.read_csv(EXPORT_DIR / "productions_for_django.csv")

for name, df in [
    ("edition", editions),
    ("translation", translations),
    ("preface", prefaces),
    ("mention", mentions),
    ("production", productions),
]:
    print(f"{name}: {len(df)} Zeilen")


edition: 327 Zeilen
translation: 19 Zeilen
preface: 272 Zeilen
mention: 1867 Zeilen
production: 667 Zeilen


In [47]:
books = pd.read_csv(EXPORT_DIR / "books_for_django.csv")
persons = pd.read_csv(EXPORT_DIR / "persons_for_django.csv")
print("books:", len(books), "persons:", len(persons))


books: 525 persons: 1980


 ## Spaltenübersicht & Stichprobe je Bundle

In [48]:
for name, df in [
    ("edition", editions),
    ("translation", translations),
    ("preface", prefaces),
    ("mention", mentions),
    ("production", productions),
]:
    print("\n====", name, "====")
    print("Spalten:", list(df.columns))
    display(df.head(3))



==== edition ====
Spalten: ['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid', 'book_target_id', 'edition_changes', 'edition_changes_format', 'edition_city_tid', 'edition_references', 'edition_references_format', 'edition_year', 'edition_year_format']


,nid,vid,type,title,status,created,changed,uid,book_target_id,edition_changes,edition_changes_format,edition_city_tid,edition_references,edition_references_format,edition_year,edition_year_format
0,2507,2507,edition,An die Herausgeber des hebräischen Sammlers üb...,1,1428490248,1428490248,1,1956,2. verbesserte und vermehrte Auflage [Voß und ...,NaN,14.0,NaN,NaN,1788,NaN
1,2508,2508,edition,Etwas zur Charackteristick der Juden - 1994,1,1428490248,1428490248,1,1994,NaN,NaN,NaN,NaN,NaN,1994,NaN
2,2509,2509,edition,Gebete der hochdeutschen und polnischen Juden ...,1,1428490248,1428490248,1,1997,NaN,NaN,14.0,NaN,NaN,1799,NaN



==== translation ====
Spalten: ['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid', 'book_target_id', 'translation_city_tid', 'translation_references', 'translation_references_format', 'translation_year', 'translation_year_format', 'translator_target_id']


,nid,vid,type,title,status,created,changed,uid,book_target_id,translation_city_tid,translation_references,translation_references_format,translation_year,translation_year_format,translator_target_id
0,2488,2488,translation,Etwas zur Charackteristick der Juden - Hebrew,1,1428490248,1428490248,1,1994,NaN,NaN,NaN,1994,NaN,NaN
1,2489,2489,translation,בית הספר א - גרמנית,1,1428490248,1428490248,1,2107,273.0,NaN,NaN,1842,NaN,1668.0
2,2490,2490,translation,בית הספר א - איטלקית,1,1428490248,1428490248,1,2107,273.0,NaN,NaN,1825,NaN,1336.0



==== preface ====
Spalten: ['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid', 'preface_notes', 'preface_notes_format', 'preface_number', 'preface_number_format', 'preface_title', 'preface_title_format', 'preface_writer_target_id']


,nid,vid,type,title,status,created,changed,uid,preface_notes,preface_notes_format,preface_number,preface_number_format,preface_title,preface_title_format,preface_writer_target_id
0,5373,5373,preface,Abhandlung von der Freiheit des Menschen - 2,1,1428490332,1428490332,1,NaN,NaN,2.0,NaN,Einleitung,filtered_html,14.0
1,5374,5374,preface,Akten-Stücke die Reform der Jüdischen Koloniee...,1,1428490332,1428490332,1,Einleitung,filtered_html,0.0,NaN,Einleitung,filtered_html,1947.0
2,5375,5375,preface,Anfangsgründe der Newtonischen Philosophie - 1,1,1428490332,1428490332,1,NaN,NaN,1.0,NaN,Vorrede,filtered_html,348.0



==== mention ====
Spalten: ['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid', 'mentionee_target_id', 'mentionee_city_tid', 'mentionee_description_tid']


,nid,vid,type,title,status,created,changed,uid,mentionee_target_id,mentionee_city_tid,mentionee_description_tid
0,2834,2834,mention,"Alexander, Levy - Answer to Mr. Joshua Van Ove...",1,1428490257,1428490257,1,15,NaN,798.0
1,2835,2835,mention,"Hoym, Karl Heinrich Graf von (Graf, Minister) ...",1,1428490257,1428490257,1,234,NaN,799.0
2,2836,2836,mention,"Gregoire, Henri - Appel à la justice des natio...",1,1428490257,1428490257,1,189,NaN,800.0



==== production ====
Spalten: ['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid', 'name_in_book', 'name_in_book_format', 'person_name_appear', 'producer_target_id', 'role_tid']


,nid,vid,type,title,status,created,changed,uid,name_in_book,name_in_book_format,person_name_appear,producer_target_id,role_tid
0,4706,4706,production,"Alexander, Isaak - אלכסנדר, יצחק - Abhandlung ...",1,1428490312,1428490312,1,NaN,NaN,NaN,14,402.0
1,4707,4707,production,Anonym - Actenstücke zur Geschichte der Erhebu...,1,1428490312,1428490312,1,NaN,NaN,NaN,17,403.0
2,4708,4708,production,"Friedländer, David - פרידלענדר, דוד - Akten-St...",1,1428490312,1428490312,1,NaN,NaN,NaN,140,402.0


## Wichtigste Referenzen checken

## Books

In [51]:
[col for col in editions.columns if "book" in col.lower()]

['book_target_id']

In [52]:
editions["book_target_id"].isna().mean()


np.float64(0.0)

In [53]:
book_ids = set(books["nid"])
missing_books = set(editions["book_target_id"].dropna()) - book_ids
len(missing_books), list(sorted(missing_books))[:10]


(0, [])

## Translations

In [54]:
[col for col in translations.columns if "book" in col.lower()]
[col for col in translations.columns if "translator" in col.lower()]


['translator_target_id']

In [55]:
translations["book_target_id"].isna().mean()


np.float64(0.0)

In [56]:
translator_cols = [c for c in translations.columns if "translator" in c.lower()]
translator_cols


['translator_target_id']

In [57]:
translator_ids = set(translations["translator_target_id"].dropna())
missing_persons = translator_ids - set(persons["nid"])
len(missing_persons), list(sorted(missing_persons))[:10]


(0, [])

## Prefaces

In [59]:
[col for col in prefaces.columns if "book" in col.lower()]
[col for col in prefaces.columns if "writer" in col.lower() or "author" in col.lower()]


['preface_writer_target_id']

## Mentions

In [61]:
[col for col in mentions.columns if "book" in col.lower()]
[col for col in mentions.columns if "mentionee" in col.lower()]
[col for col in mentions.columns if "city" in col.lower()]


['mentionee_city_tid']

In [62]:
mentions["book_target_id"].isna().mean()
mentions["mentionee_target_id"].isna().mean()


KeyError: 'book_target_id'

## Productions

In [64]:
[col for col in productions.columns if "book" in col.lower()]
[col for col in productions.columns if "producer" in col.lower() or "person" in col.lower()]
[col for col in productions.columns if "role" in col.lower()]


['role_tid']

In [66]:
productions["book_target_id"].isna().mean()


np.float64(0.0)

In [69]:
print(editions.columns)
print(translations.columns)
print(prefaces.columns)
print(mentions.columns)
print(productions.columns)

Index(['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid',
       'book_target_id', 'edition_changes', 'edition_changes_format',
       'edition_city_tid', 'edition_references', 'edition_references_format',
       'edition_year', 'edition_year_format'],
      dtype='object')
Index(['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid',
       'book_target_id', 'translation_city_tid', 'translation_references',
       'translation_references_format', 'translation_year',
       'translation_year_format', 'translator_target_id'],
      dtype='object')
Index(['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid',
       'preface_notes', 'preface_notes_format', 'preface_number',
       'preface_number_format', 'preface_title', 'preface_title_format',
       'preface_writer_target_id'],
      dtype='object')
Index(['nid', 'vid', 'type', 'title', 'status', 'created', 'changed', 'uid',
       'mentionee_target_id', 'mentionee_city_tid',
       'me

## Sanity Check

In [70]:
for name, df in [
    ("edition", editions),
    ("translation", translations),
    ("preface", prefaces),
    ("mention", mentions),
    ("production", productions),
]:
    print("\n====", name, "nicht-node-Spalten ====")
    print([c for c in df.columns if c not in ("nid", "vid", "type", "title", "status", "created", "changed", "uid")])



==== edition nicht-node-Spalten ====
['book_target_id', 'edition_changes', 'edition_changes_format', 'edition_city_tid', 'edition_references', 'edition_references_format', 'edition_year', 'edition_year_format']

==== translation nicht-node-Spalten ====
['book_target_id', 'translation_city_tid', 'translation_references', 'translation_references_format', 'translation_year', 'translation_year_format', 'translator_target_id']

==== preface nicht-node-Spalten ====
['preface_notes', 'preface_notes_format', 'preface_number', 'preface_number_format', 'preface_title', 'preface_title_format', 'preface_writer_target_id']

==== mention nicht-node-Spalten ====
['mentionee_target_id', 'mentionee_city_tid', 'mentionee_description_tid']

==== production nicht-node-Spalten ====
['name_in_book', 'name_in_book_format', 'person_name_appear', 'producer_target_id', 'role_tid']


In [71]:
# Node-Referenzen von Books zu anderen Bundles
book_relations = bundle_field_meta[
    (bundle_field_meta['bundle'] == 'book') &
    (bundle_field_meta['field_kind'] == 'node')
][['field_base', 'field_column', 'storage_table', 'target_bundles']].drop_duplicates()

book_relations


,field_base,field_column,storage_table,target_bundles
151,field_old_text_author,field_old_text_author_target_id,field_data_field_old_text_author,person
169,field_original_text_author,field_original_text_author_target_id,field_data_field_original_text_author,person


In [72]:
[c for c in books.columns if "preface" in c.lower() or "mention" in c.lower() or "product" in c.lower()]


['mention_general_notes',
 'mention_general_notes_format',
 'mentions_in_reviews',
 'mentions_in_reviews_format',
 'original_sources_mention',
 'original_sources_mention_format',
 'preface',
 'production_evidence',
 'production_evidence_format',
 'sources_not_mentioned',
 'sources_not_mentioned_list',
 'sources_not_mentioned_list_format',
 'sources_not_mentioned_ref',
 'sources_not_mentioned_ref_format',
 'structure_preface_notes',
 'structure_preface_notes_format']

## Search for hidden linkers

In [73]:
import pandas as pd

# Alle field_data_field_* Tabellen, die *_target_id-Spalten haben:
node_ref_tables = bundle_field_meta[
    (bundle_field_meta['field_kind'] == 'node')
][['storage_table', 'field_column', 'bundle', 'target_bundles']].drop_duplicates()

node_ref_tables.sort_values(['storage_table', 'bundle'])


,storage_table,field_column,bundle,target_bundles
10,field_data_field_book,field_book_target_id,edition,book
9,field_data_field_book,field_book_target_id,translation,book
117,field_data_field_mentionee,field_mentionee_target_id,mention,person
151,field_data_field_old_text_author,field_old_text_author_target_id,book,person
169,field_data_field_original_text_author,field_original_text_author_target_id,book,person
206,field_data_field_preface_writer,field_preface_writer_target_id,preface,person
234,field_data_field_producer,field_producer_target_id,production,person
313,field_data_field_translator,field_translator_target_id,translation,person


## Taxonomy Export

In [79]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(exist_ok=True)

def export_taxonomy_simple(vid: int, slug: str):
    """
    Exportiert alle Terms eines Vokabulars (nur taxonomy_term_data)
    nach export/taxonomy_<slug>.csv
    """
    print(f"\n=== Export Taxonomie vid={vid} ({slug}) ===")

    sql = f"""
    SELECT tid, vid, name, description, weight, format
    FROM taxonomy_term_data
    WHERE vid = {vid}
    """
    df = pd.read_sql(sql, con=db_connection)

    if df.empty:
        print(f"⚠️ Keine Begriffe für vid={vid} gefunden.")
        return

    print(f"  {len(df)} Begriffe gefunden.")

    out_path = EXPORT_DIR / f"taxonomy_{slug}.csv"
    df.to_csv(out_path, index=False)
    print(f"✅ Export geschrieben nach: {out_path}")


In [80]:
export_taxonomy_simple(2,  "language_counts")
export_taxonomy_simple(3,  "models")
export_taxonomy_simple(4,  "description_of_mentionee")
export_taxonomy_simple(5,  "gender")
export_taxonomy_simple(6,  "occupation")
export_taxonomy_simple(7,  "alignment_of_text")
export_taxonomy_simple(8,  "date_format")
export_taxonomy_simple(9,  "type_of_edition")
export_taxonomy_simple(10, "fonts")
export_taxonomy_simple(11, "languages")
export_taxonomy_simple(14, "publishers")
export_taxonomy_simple(16, "series")
export_taxonomy_simple(17, "target_audience")
export_taxonomy_simple(18, "topics")
export_taxonomy_simple(19, "typography")



=== Export Taxonomie vid=2 (language_counts) ===
  4 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_language_counts.csv

=== Export Taxonomie vid=3 (models) ===
  71 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_models.csv

=== Export Taxonomie vid=4 (description_of_mentionee) ===
  11 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_description_of_mentionee.csv

=== Export Taxonomie vid=5 (gender) ===
  1 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_gender.csv

=== Export Taxonomie vid=6 (occupation) ===
  6 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_occupation.csv

=== Export Taxonomie vid=7 (alignment_of_text) ===
  3 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_alignment_of_text.csv

=== Export Taxonomie vid=8 (date_format) ===
  4 Begriffe gefunden.
✅ Export geschrieben nach: export/taxonomy_date_format.csv

=== Export Taxonomie vid=9 (type_of_edition) ===
  8 Begriffe gefunden.
✅ Exp

## Hilfstabllen

In [93]:
from sqlalchemy import create_engine
import pandas as pd

db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/haskala_backup'
db_connection = create_engine(db_connection_str)

In [94]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(exist_ok=True)

def export_whole_table(table_name: str, slug: str | None = None):
    """
    Exportiert eine komplette Tabelle als CSV:
    SELECT * FROM <table_name>
    """
    if slug is None:
        slug = table_name

    print(f"\n=== Export Tabelle {table_name} ===")

    df = pd.read_sql(f"SELECT * FROM {table_name}", con=db_connection)
    print(f"  {len(df)} Zeilen, {len(df.columns)} Spalten.")

    out_path = EXPORT_DIR / f"{slug}.csv"
    df.to_csv(out_path, index=False)
    print(f"✅ Export geschrieben nach: {out_path}")


In [95]:
export_whole_table("taxonomy_index", "taxonomy_index")
export_whole_table("file_managed", "file_managed")
export_whole_table("users", "users")



=== Export Tabelle taxonomy_index ===
  12767 Zeilen, 4 Spalten.
✅ Export geschrieben nach: export/taxonomy_index.csv

=== Export Tabelle file_managed ===
  0 Zeilen, 8 Spalten.
✅ Export geschrieben nach: export/file_managed.csv

=== Export Tabelle users ===
  4 Zeilen, 16 Spalten.
✅ Export geschrieben nach: export/users.csv


In [86]:
pd.read_sql("SHOW TABLES", con=db_connection)


,Tables_in_haskala
0,field_config
1,field_config_instance
2,field_data_field_alignment
3,field_data_field_bans
4,field_data_field_bar_ilan_library_id
...,...
417,sequences
418,system
419,taxonomy_index
420,taxonomy_term_data


In [96]:
export_whole_table("users_roles", "users_roles")        # falls vorhanden
export_whole_table("role", "role")                      # Rollen-Definitionen



=== Export Tabelle users_roles ===
  3 Zeilen, 2 Spalten.
✅ Export geschrieben nach: export/users_roles.csv

=== Export Tabelle role ===
  4 Zeilen, 3 Spalten.
✅ Export geschrieben nach: export/role.csv


In [97]:
pd.read_sql("SHOW TABLES LIKE 'languages'", con=db_connection)


,Tables_in_haskala_backup (languages)


In [99]:
export_whole_table("url_alias", "url_alias")



=== Export Tabelle url_alias ===
  6470 Zeilen, 4 Spalten.
✅ Export geschrieben nach: export/url_alias.csv


In [100]:
export_whole_table("variable", "variable")



=== Export Tabelle variable ===
  150 Zeilen, 2 Spalten.
✅ Export geschrieben nach: export/variable.csv


In [101]:
vars_df = pd.read_sql("SELECT name, value FROM variable", con=db_connection)

vars_df[vars_df['name'].str.contains('pathauto', na=False)]


,name,value
128,pathauto_blog_pattern,"b's:17:""blogs/[user:name]"";'"
129,pathauto_forum_pattern,"b's:29:""[term:vocabulary]/[term:name]"";'"
130,pathauto_node_book_pattern,"b's:18:""books/[node:title]"";'"
131,pathauto_node_pattern,"b's:20:""content/[node:title]"";'"
132,pathauto_node_person_pattern,"b's:19:""people/[node:title]"";'"
133,pathauto_punctuation_hyphen,b'i:1;'
134,pathauto_taxonomy_term_pattern,"b's:29:""[term:vocabulary]/[term:name]"";'"
135,pathauto_user_pattern,"b's:17:""users/[user:name]"";'"


In [104]:
import pandas as pd


aliases = pd.read_sql(
    "SELECT pid, source, alias, language FROM url_alias",
    con=db_connection,
)

aliases.head()


,pid,source,alias,language
0,1,user/1,users/admin,und
1,2,taxonomy/term/1,cities/altona,und
2,3,taxonomy/term/2,cities/altstadt,und
3,4,taxonomy/term/3,cities/amsterdam,und
4,5,taxonomy/term/4,cities/anklam,und


In [105]:
aliases_preface = aliases[aliases['alias'].str.contains("preface", na=False)]
aliases_mention = aliases[aliases['alias'].str.contains("mention", na=False)]
aliases_production = aliases[aliases['alias'].str.contains("production", na=False)]

aliases_preface.head(20)
aliases_mention.head(20)
aliases_production.head(20)


,pid,source,alias,language


In [106]:
# Alle Preface-/Mention-/Production-NIDs
preface_nodes = pd.read_sql("SELECT nid, type, title FROM node WHERE type='preface'", con=db_connection)
mention_nodes = pd.read_sql("SELECT nid, type, title FROM node WHERE type='mention'", con=db_connection)
production_nodes = pd.read_sql("SELECT nid, type, title FROM node WHERE type='production'", con=db_connection)
book_nodes = pd.read_sql("SELECT nid, type, title FROM node WHERE type='book'", con=db_connection)
person_nodes = pd.read_sql("SELECT nid, type, title FROM node WHERE type='person'", con=db_connection)


In [107]:
def add_alias(df_nodes, label):
    df = df_nodes.copy()
    df['source'] = 'node/' + df['nid'].astype(str)
    df = df.merge(aliases, on='source', how='left')
    print(f"\n=== {label} ({len(df)}) ===")
    print(df[['nid', 'title', 'alias']].head(10))
    return df

books_with_alias      = add_alias(book_nodes,      "Books")
persons_with_alias    = add_alias(person_nodes,    "Persons")
prefaces_with_alias   = add_alias(preface_nodes,   "Prefaces")
mentions_with_alias   = add_alias(mention_nodes,   "Mentions")
productions_with_alias = add_alias(production_nodes, "Productions")



=== Books (525) ===
    nid                                              title  \
0  1952           Abhandlung von der Freiheit des Menschen   
1  1953  Actenstücke zur Geschichte der Erhebung der Ju...   
2  1954  Akten-Stücke die Reform der Jüdischen Koloniee...   
3  1955                          An das Breslauer Publikum   
4  1956  An die Herausgeber des hebräischen Sammlers üb...   
5  1958         Anfangsgründe der Newtonischen Philosophie   
6  1959  Answer to Mr. Joshua Van Oven’s letters on the...   
7  1960  Apologie der Menschenrechte oder philosophisch...   
8  1961        Apologie des juifs en réponse à la question   
9  1962         Appel à la justice des nations et des rois   

                                               alias  
0     books/abhandlung-von-der-freiheit-des-menschen  
1  books/actenstücke-zur-geschichte-der-erhebung-...  
2  books/akten-stücke-die-reform-der-jüdischen-ko...  
3                       books/das-breslauer-publikum  
4  books/die-herausge

In [114]:
export_whole_table("field_config", "field_config")
export_whole_table("field_config_instance", "field_config_instance")



=== Export Tabelle field_config ===
  208 Zeilen, 13 Spalten.
✅ Export geschrieben nach: export/field_config.csv

=== Export Tabelle field_config_instance ===
  221 Zeilen, 7 Spalten.
✅ Export geschrieben nach: export/field_config_instance.csv


In [115]:
export_whole_table("menu_links", "menu_links")
export_whole_table("menu_router", "menu_router")



=== Export Tabelle menu_links ===
  332 Zeilen, 25 Spalten.
✅ Export geschrieben nach: export/menu_links.csv

=== Export Tabelle menu_router ===
  400 Zeilen, 23 Spalten.
✅ Export geschrieben nach: export/menu_router.csv


In [116]:
export_whole_table("node_type", "node_type")



=== Export Tabelle node_type ===
  8 Zeilen, 13 Spalten.
✅ Export geschrieben nach: export/node_type.csv


In [117]:
export_whole_table("role", "role")
export_whole_table("role_permission", "role_permission")



=== Export Tabelle role ===
  4 Zeilen, 3 Spalten.
✅ Export geschrieben nach: export/role.csv

=== Export Tabelle role_permission ===
  214 Zeilen, 3 Spalten.
✅ Export geschrieben nach: export/role_permission.csv


## Revisions

In [118]:
export_whole_table("node_revision", "node_revision")



=== Export Tabelle node_revision ===
  5658 Zeilen, 10 Spalten.
✅ Export geschrieben nach: export/node_revision.csv


In [125]:
import pandas as pd

rev_tables = pd.read_sql(
    """
    SELECT TABLE_NAME
    FROM information_schema.tables
    WHERE table_schema = 'haskala'
      AND TABLE_NAME LIKE 'field_revision_field_%%'
    ORDER BY TABLE_NAME;
    """,
    con=db_connection,
)

rev_tables


,TABLE_NAME
0,field_revision_field_alignment
1,field_revision_field_bans
2,field_revision_field_bar_ilan_library_id
3,field_revision_field_berlin_library_id
4,field_revision_field_bibliographical_citations
...,...
190,field_revision_field_typography
191,field_revision_field_viaf_id
192,field_revision_field_volumes_notes
193,field_revision_field_volumes_published_number


In [126]:
rev_list = rev_tables["TABLE_NAME"].tolist()
rev_list


['field_revision_field_alignment',
 'field_revision_field_bans',
 'field_revision_field_bar_ilan_library_id',
 'field_revision_field_berlin_library_id',
 'field_revision_field_bibliographical_citations',
 'field_revision_field_book',
 'field_revision_field_book_availability_notes',
 'field_revision_field_book_not_available',
 'field_revision_field_book_structure_notes',
 'field_revision_field_book_studies',
 'field_revision_field_book_type_general_notes',
 'field_revision_field_british_library_id',
 'field_revision_field_catalog_numbers_notes',
 'field_revision_field_censorship',
 'field_revision_field_contacts_official_agents',
 'field_revision_field_contacts_other_people',
 'field_revision_field_contemporary_disputes',
 'field_revision_field_contemporary_references',
 'field_revision_field_content',
 'field_revision_field_contents_table_notes',
 'field_revision_field_contradict_new_edition',
 'field_revision_field_contradict_original',
 'field_revision_field_copy_of_book_used',
 'fie

In [127]:
for tbl in rev_list:
    export_whole_table(tbl, tbl)



=== Export Tabelle field_revision_field_alignment ===
  454 Zeilen, 8 Spalten.
✅ Export geschrieben nach: export/field_revision_field_alignment.csv

=== Export Tabelle field_revision_field_bans ===
  3 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bans.csv

=== Export Tabelle field_revision_field_bar_ilan_library_id ===
  66 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bar_ilan_library_id.csv

=== Export Tabelle field_revision_field_berlin_library_id ===
  19 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_berlin_library_id.csv

=== Export Tabelle field_revision_field_bibliographical_citations ===
  1 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bibliographical_citations.csv

=== Export Tabelle field_revision_field_book ===
  3152 Zeilen, 8 Spalten.
✅ Export geschrieben nach: export/field_revision_field_book.csv

=== Export Tabelle field_revision_field_book_availability_note

In [129]:
all_rev = pd.read_sql(
    """
    SELECT TABLE_NAME
    FROM information_schema.tables
    WHERE table_schema = 'haskala'
      AND TABLE_NAME LIKE '%%revision%%';
    """,
    con=db_connection,
)

all_rev


,TABLE_NAME
0,field_revision_field_alignment
1,field_revision_field_bans
2,field_revision_field_bar_ilan_library_id
3,field_revision_field_berlin_library_id
4,field_revision_field_bibliographical_citations
...,...
191,field_revision_field_viaf_id
192,field_revision_field_volumes_notes
193,field_revision_field_volumes_published_number
194,field_revision_field_width


In [130]:
for tbl in rev_list:
    export_whole_table(tbl, tbl)



=== Export Tabelle field_revision_field_alignment ===
  454 Zeilen, 8 Spalten.
✅ Export geschrieben nach: export/field_revision_field_alignment.csv

=== Export Tabelle field_revision_field_bans ===
  3 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bans.csv

=== Export Tabelle field_revision_field_bar_ilan_library_id ===
  66 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bar_ilan_library_id.csv

=== Export Tabelle field_revision_field_berlin_library_id ===
  19 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_berlin_library_id.csv

=== Export Tabelle field_revision_field_bibliographical_citations ===
  1 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/field_revision_field_bibliographical_citations.csv

=== Export Tabelle field_revision_field_book ===
  3152 Zeilen, 8 Spalten.
✅ Export geschrieben nach: export/field_revision_field_book.csv

=== Export Tabelle field_revision_field_book_availability_note

## Extra eports

In [142]:
export_whole_table("field_data_field_location_of_footnotes", "location_of_footnotes")
export_whole_table("field_data_field_language_of_footnotes", "language_of_footnotes")


=== Export Tabelle field_data_field_location_of_footnotes ===
  182 Zeilen, 8 Spalten.
✅ Export geschrieben nach: research/export/location_of_footnotes.csv

=== Export Tabelle field_data_field_language_of_footnotes ===
  205 Zeilen, 8 Spalten.
✅ Export geschrieben nach: research/export/language_of_footnotes.csv


In [146]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

def export_footnote_locations():
    """
    Exportiert alle verwendeten Footnote-Locations:
    - Quelle: field_data_field_location_of_footnotes + taxonomy_term_data
    - Ziel:  research/export/taxonomy_footnote_locations.csv
    """
    sql = """
    SELECT DISTINCT
        t.tid,
        t.vid,
        t.name,
        t.description
    FROM field_data_field_location_of_footnotes f
    JOIN taxonomy_term_data t
      ON f.field_location_of_footnotes_tid = t.tid
    ORDER BY t.tid;
    """

    df = pd.read_sql(sql, con=db_connection)
    out_path = EXPORT_DIR / "taxonomy_footnote_locations.csv"
    df.to_csv(out_path, index=False)
    print(f"Footnote-Locations: {len(df)} Zeilen -> {out_path}")

export_footnote_locations()


Footnote-Locations: 3 Zeilen -> export/taxonomy_footnote_locations.csv


In [137]:
def export_field_relation(table, field_col, out_name):
    sql = f"""
        SELECT entity_id AS nid, {field_col} AS tid
        FROM {table}
        ORDER BY entity_id, delta
    """
    df = pd.read_sql(sql, con=db_connection)
    path = f"export/{out_name}.csv"
    df.to_csv(path, index=False)
    print(f"Exported {len(df)} rows → {path}")


In [138]:
export_field_relation(
    "field_data_field_main_textual_models",
    "field_main_textual_models_tid",
    "main_textual_models"
)

export_field_relation(
    "field_data_field_secondary_textual_models",
    "field_secondary_textual_models_tid",
    "secondary_textual_models"
)

export_field_relation(
    "field_data_field_alignment",
    "field_alignment_tid",
    "alignment_relations"
)


Exported 624 rows → export/main_textual_models.csv
Exported 421 rows → export/secondary_textual_models.csv
Exported 454 rows → export/alignment_relations.csv


In [139]:
export_whole_table("field_data_field_textual_models_notes", "textual_models_notes")



=== Export Tabelle field_data_field_textual_models_notes ===
  147 Zeilen, 9 Spalten.
✅ Export geschrieben nach: export/textual_models_notes.csv


In [147]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

def export_textual_models_and_alignment():
    """
    Exportiert:
    - Textual Models  (vid = 3)    -> taxonomy_textual_models.csv
    - Alignment       (vid = 7)    -> taxonomy_alignment.csv
    """
    mapping = [
        (3, "textual_models"),
        (7, "alignment"),
    ]

    for vid, slug in mapping:
        sql = f"""
        SELECT tid, vid, name, description
        FROM taxonomy_term_data
        WHERE vid = {vid}
        ORDER BY tid;
        """
        df = pd.read_sql(sql, con=db_connection)
        out_path = EXPORT_DIR / f"taxonomy_{slug}.csv"
        df.to_csv(out_path, index=False)
        print(f"{slug}: {len(df)} Zeilen -> {out_path}")

export_textual_models_and_alignment()


textual_models: 71 Zeilen -> export/taxonomy_textual_models.csv
alignment: 3 Zeilen -> export/taxonomy_alignment.csv


### Original Types

In [149]:
import pandas as pd
from pathlib import Path

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

def export_original_type():
    """
    Exportiert alle verwendeten OriginalType-Taxonomiebegriffe:
    - Quelle: field_data_field_original_type + taxonomy_term_data
    - Ziel:  research/export/taxonomy_original_type.csv
    """
    sql = """
    SELECT DISTINCT
        t.tid,
        t.vid,
        t.name,
        t.description
    FROM field_data_field_original_type f
    JOIN taxonomy_term_data t
      ON f.field_original_type_tid = t.tid
    ORDER BY t.tid;
    """

    df = pd.read_sql(sql, con=db_connection)
    out_path = EXPORT_DIR / "taxonomy_original_type.csv"
    df.to_csv(out_path, index=False)
    print(f"OriginalType: {len(df)} Zeilen -> {out_path}")

export_original_type()


OriginalType: 3 Zeilen -> export/taxonomy_original_type.csv


### Textual Models

In [150]:
def export_textual_models():
    """
    Exportiert alle verwendeten Textual Models (main + secondary):
    - Quelle: field_data_field_main_textual_models,
              field_data_field_secondary_textual_models,
              taxonomy_term_data
    - Ziel:   research/export/taxonomy_textual_models.csv
    """
    sql = """
    SELECT DISTINCT
        t.tid,
        t.vid,
        t.name,
        t.description
    FROM (
        SELECT field_main_textual_models_tid AS tid
        FROM field_data_field_main_textual_models
        UNION
        SELECT field_secondary_textual_models_tid AS tid
        FROM field_data_field_secondary_textual_models
    ) x
    JOIN taxonomy_term_data t
      ON t.tid = x.tid
    ORDER BY t.tid;
    """

    df = pd.read_sql(sql, con=db_connection)
    out_path = EXPORT_DIR / "taxonomy_textual_models.csv"
    df.to_csv(out_path, index=False)
    print(f"Textual models: {len(df)} Zeilen -> {out_path}")

export_textual_models()


Textual models: 62 Zeilen -> export/taxonomy_textual_models.csv


# Ontology


# How the database works:

The most important tables are:
node - contains the nodes:
 The id in nodes is the nid.
 # We leave ot the vid for now. Don't know what it is used for,
node_revision - contains the revisions of the nodes

mentionee


# export column 'field_data_field_content' as text file
there is only one cell.


In [49]:
field_data_field_content = pd.read_sql('SELECT * FROM field_data_field_content', con=db_connection)
content = field_data_field_content.loc[0]['field_content_value']

# write the content to a file
with open('content.txt', 'wb') as f:
    f.write(content.encode('utf-8'))

print(content)

<p>This database contains 525 books in German and Hebrew identified by leading scholars as Haskala literature. The database is hosted and run by the Judaica Division of the Frankfurt University Library.</p>
<p>Contact: (E-Mail: digitalisierung@ub.uni-frankfurt.de / r.heuberger@ub.uni-frankfurt.de / u.simeon@ub.uni-frankfurt.de )	Please share any further information.</p>
<p>The detailed bio-bibliographical information, presented here for the first	time, was collected as result of a collaborative research, headed by Profs.	Shmuel Feiner, Bar Ilan University; Zohar Shavit, Tel Aviv University and	Christoph Schulte, Universität Potsdam within a joint German-Israeli Research	Project in 2004-2010.</p>
<p>Please note: for every book title only those fields containing information	are presented.</p>



In [50]:
# Get the nodes
haskala_node = pd.read_sql('SELECT * FROM node', con=db_connection)
haskala_node.head(5)


,nid,vid,type,language,title,uid,status,created,changed,comment,promote,sticky,tnid,translate
0,1,1,page_element,en,Introduction,1,1,1428490114,1428490114,0,0,0,0,0
1,2,2,person,und,(Dr. med.) איסטרייכער,1,1,1428490114,1428490114,0,1,0,0,0
2,3,3,person,und,(Dr. med.) ווארבורג,1,1,1428490114,1428490114,0,1,0,0,0
3,4,4,person,und,(Dr.) בריינערסדארף,1,1,1428490114,1428490114,0,1,0,0,0
4,5,5,person,und,"(Feder)-Guttmann, Tuvia - פדר, טוביה",1,1,1428490114,1428490114,0,1,0,0,0


# We leave out the revision fields for now.
# Todo: add the revision fields


In [ ]:
# get the 'data_field_full_title' fields
haskala_field_data_field_full_title = pd.read_sql('SELECT * FROM field_data_field_full_title', con=db_connection)
haskala_field_data_field_full_title.head(5)


In [ ]:
# match the nodes columns of entity_id the field_data_field_full_title and nid form the node table
haskala_title = haskala_node.merge(haskala_field_data_field_full_title, left_on='nid', right_on='entity_id')
haskala_title.head(5)


# Persons
## Get person data

nid
vid
uid
status

entity_id_nid


In [ ]:
haskala_person = haskala_node[haskala_node['type'] == 'person']

haskala_gender = pd.read_sql('SELECT * FROM field_data_field_gender', con=db_connection)
haskala_person = haskala_person.join(haskala_gender, on='nid', how='left', lsuffix='_nid')

haskala_date_of_birth = pd.read_sql('SELECT * FROM field_data_field_date_of_birth', con=db_connection)
haskala_person = haskala_person.join(haskala_date_of_birth, on='nid', how='left', lsuffix='_nid')

haskala_date_of_death = pd.read_sql('SELECT * FROM field_data_field_date_of_death', con=db_connection)
haskala_person = haskala_person.join(haskala_date_of_death, on='nid', how='left', lsuffix='_nid')

haskala_occupation = pd.read_sql('SELECT * FROM field_data_field_occupation', con=db_connection)
haskala_person = haskala_person.join(haskala_occupation, on='nid', how='left', lsuffix='_nid')


haskala_person.head(10)

## Get the taxonomy
The table taxonomy_term_data contains the taxonomy terms.
It contains names of places, gender, ...

In [ ]:
taxonomy_term_data = pd.read_sql('SELECT * FROM taxonomy_term_data', con=db_connection)
taxonomy_term_data.head(5)

## Match the occupation
occupation is only a value and needs to be matched in the database

In [ ]:
## Match the occupation
haskala_person = haskala_person.merge(taxonomy_term_data, left_on='field_occupation_tid', right_on='tid', how='left')
haskala_person['field_occupation_tid'] = haskala_person['name'].fillna(haskala_person['field_occupation_tid'])
haskala_person.sample(10)

## Get the VIAF ID

In [ ]:
haskala_field_data_field_viaf_id = pd.read_sql('SELECT * FROM field_data_field_viaf_id', con=db_connection)
haskala_field_data_field_viaf_id.head(5)

# add the VIAF ID to the person table
haskala_person = haskala_person.join(haskala_field_data_field_viaf_id, on='entity_id', how='left', lsuffix='_nid')
haskala_person.head(5)

## Match the gender

In [ ]:
# Replace the  gender ID with the according tid from the taxonomy_term_data table.
# convert field_gender_tid to integer.
#haskala_person['field_gender_tid'] = haskala_person['field_gender_tid'].astype(int)
haskala_person = haskala_person.merge(taxonomy_term_data, left_on='field_gender_tid', right_on='tid')
haskala_person['field_gender'] = haskala_person['name']
#haskala_person = haskala_person.join(taxonomy_term_data, on='field_gender_tid', how='left', lsuffix='tid')
haskala_person.sample(10)

## Add Hebrew name

In [ ]:
# add the hebrew name from the table field_data_field_hebrew_name
haskala_field_data_field_hebrew_name = pd.read_sql('SELECT * FROM field_data_field_hebrew_name', con=db_connection)
haskala_person = haskala_person.join(haskala_field_data_field_hebrew_name, on='entity_id', how='left', lsuffix='_nid')
haskala_person.sample(10)

## Add German name

In [ ]:
# add the hebrew name from the table field_data_field_hebrew_name
haskala_field_data_field_german_name = pd.read_sql('SELECT * FROM field_data_field_german_name', con=db_connection)
haskala_person = haskala_person.join(haskala_field_data_field_german_name, on='entity_id', how='left', lsuffix='_nid')
haskala_person.sample(10)

## Build the Graph for the persons

In [ ]:
file_name = 'haskala-persons'
working_path = "./"
output_path = "data/"

graph = Graph()

skos = Namespace("http://www.w3.org/2004/02/skos/core#")
jl = Namespace("http://data.judaicalink.org/ontology/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
dc = Namespace("http://purl.org/dc/elements/1.1/")

graph.bind('skos', skos)
graph.bind('foaf', foaf)
graph.bind('jl', jl)
graph.bind('gndo', gndo)
graph.bind('owl', owl)
graph.bind('edm', edm)
graph.bind('dc', dc)

## Clean url string

In [ ]:
def clean_url_string(string):
    """
    Clean the name of a person.
    returns: cleaned name.
    """

    string = string.strip()
    string = string.replace('\'', '')
    string = string.replace('"', '')
    string = string.replace(',', '_')
    string = string.replace('<<', '')
    string = string.replace('>>', '')
    string = string.replace('|', '_')
    string = string.replace(' ', '')
    string = string.replace('<', '_')
    string = string.replace('>', '_')
    string = string.replace('.', '')
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace('(', '')
    string = string.replace(')', '')
    string = string.replace('{', '')
    string = string.replace('}', '')
    return string


## Generate RDF for the persons

In [ ]:
"""
Generate RDF for the persons.
returns: nothing
"""

skos = Namespace("http://www.w3.org/2004/02/skos/core#")
jl = Namespace("http://data.judaicalink.org/ontology/")
hs = Namespace("http://data.haskala-library.net/ontology/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
dc = Namespace("http://purl.org/dc/elements/1.1/")
dcterms = Namespace("http://purl.org/dc/terms/")

graph.bind('skos', skos)
graph.bind('foaf', foaf)
graph.bind('jl', jl)
graph.bind('hs', hs)
graph.bind('gndo', gndo)
graph.bind('owl', owl)
graph.bind('edm', edm)
graph.bind('dc', dc)


# iterate over the rows in the dataframe
for index, row in haskala_person.iterrows():

    url_name = clean_url_string(row['title'])
    uri = URIRef(f"http://data.judaicalink.org/data/haskala/{url_name}")
    prefLabel = row['title'].split("-")[0].strip()
    print(prefLabel)
    graph.add((URIRef(uri), foaf.name, Literal(prefLabel)))
    graph.add((URIRef(uri), skos.prefLabel, Literal(prefLabel)))

    try:
        altLabel = row['title'].split("-", ',')[1].strip()
        if  altLabel != '':
            graph.add((URIRef(uri), skos.altLabel, Literal(row['title'].split("-", ',')[1].strip())))
            print(altLabel)
    except:
        pass

    #graph.add((URIRef(uri), RDF.type, foaf.Person))

    #print(not pd.isna(row['field_date_of_birth_value']))

    if row['field_date_of_birth_value'] != '' and not pd.isna(row['field_date_of_birth_value']):
        graph.add((URIRef(uri), gndo.birthDate, Literal(row['field_date_of_birth_value'])))
    
    if row['field_date_of_death_value'] != '' and not pd.isna(row['field_date_of_death_value']):
        graph.add((URIRef(uri), gndo.deathDate, Literal(row['field_date_of_death_value'])))
    
    if row['field_gender'] != '' and not pd.isna(row['field_gender']):
        graph.add((URIRef(uri), gndo.gender, Literal(row['field_gender'])))

    
    if row['field_occupation_tid'] != '' and not pd.isna(row['field_occupation_tid']):
        for occupation in str(row['field_occupation_tid']).split(';'):
            graph.add((URIRef(uri), gndo.occupation, Literal(occupation.strip())))
    '''
    if row['hasPublication'] != '' and not pd.isna(row['hasPublication']):
        for publication in str(row['hasPublication']).split(';'):
            graph.add((URIRef(uri), gndo.hasPublication, Literal(publication.strip())))
    
    if row['alternativeName'] != '' and not pd.isna(row['alternativeName']):
        graph.add((URIRef(uri), gndo.alternativeName, Literal(row['alternativeName'])))
    
    if row['hasAbstract'] != '' and not pd.isna(row['hasAbstract']):
        graph.add((URIRef(uri), gndo.hasAbstract, Literal(row['hasAbstract'].strip())))
        
    '''
    
    if row['field_german_name_value'] != '' and not pd.isna(row['field_german_name_value']):
        graph.add((URIRef(uri), hs.hasGermanName, Literal(row['field_german_name_value'])))
        
    if row['field_hebrew_name_value'] != '' and not pd.isna(row['field_hebrew_name_value']):
        graph.add((URIRef(uri), hs.hasHebrewName, Literal(row['field_hebrew_name_value'])))
        
    if row['field_viaf_id_value'] != '' and not pd.isna(row['field_viaf_id_value']):
        graph.add((URIRef(uri), owl.sameAs, URIRef(f"http://viaf.org/viaf/{row['field_viaf_id_value']}"))) 
        graph.add((URIRef(uri), hs.hasVIAFID, Literal(row['field_viaf_id_value'])))

    #graph.add((URIRef(uri), dcterms.created, (Literal(datetime.now()))))


graph.serialize(destination=file_name + "-" + datetime.today().strftime('%Y-%m-%d') + ".ttl", format="turtle")


# Books

# Digital Books

# Places
## Extracting the places from the database
Extract all the places from the tables.
The places are listed in the following tables:
'taxonomy_term_data' with vid = 1.
vid is the category and tid is the id of the place.

## How to find the books related to the places
The tid is found the table 'taxonomy_index' with nid = the id of the place.
the nid is found in the table 'node'.

## Website
Places are sorted alphabetically by the name of the place on the website.
Markers are placed on the map according to the coordinates of the place.
If you click on the place you get the list of the books related to the place.
If you click on the link you will be redirected to the book.
Todo: add the persons to the list

## RDF


## Extraction of the places from the database

In [ ]:

haskala_place = pd.read_sql_query("SELECT * FROM taxonomy_term_data WHERE vid = 1" , con=db_connection)
haskala_place = haskala_place[['tid', 'name']]
taxonomy_index = pd.read_sql_query("SELECT * FROM taxonomy_index" , con=db_connection)
taxonomy_index = taxonomy_index[['nid', 'tid']]
haskala_place = pd.merge(haskala_place, taxonomy_index, on='tid')


In [ ]:
haskala_place.sample(5)


In [ ]:
haskala_place_books_relation = pd.read_sql_query("SELECT * FROM node" , con=db_connection)
haskala_place_books_relation = haskala_place_books_relation[['nid', 'title', 'type']]
haskala_place = pd.merge(haskala_place, haskala_place_books_relation, on='nid')

In [ ]:

haskala_place.sample(10)

## RDF Graph build for places

In [ ]:
"""
Generate RDF for the places.
returns: nothing
"""
graph = Graph()

skos = Namespace("http://www.w3.org/2004/02/skos/core#")
jl = Namespace("http://data.judaicalink.org/ontology/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
gndo = Namespace("http://d-nb.info/standards/elementset/gnd#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
dc = Namespace("http://purl.org/dc/elements/1.1/")
dcterms = Namespace("http://purl.org/dc/terms/")

graph.bind('skos', skos)
graph.bind('foaf', foaf)
graph.bind('jl', jl)
graph.bind('gndo', gndo)
graph.bind('owl', owl)
graph.bind('edm', edm)
graph.bind('dc', dc)

file_name = "haskala-places-" + datetime.today().strftime('%Y-%m-%d') + ".ttl"
if os.path.exists(file_name):
    os.remove(file_name)

# iterate over the rows in the dataframe
for index, row in haskala_place.iterrows():

    place = row['name']
    url_name = clean_url_string(place)
    uri = URIRef(f"http://data.judaicalink.org/data/haskala/{url_name}")

    graph.add((URIRef(uri), foaf.name, Literal(place)))
    graph.add((URIRef(uri), skos.prefLabel, Literal(place)))
    graph.add((URIRef(uri), RDF.type, jl.Place))

    title = row['title']
    title_url_name = clean_url_string(title)
    if row['type'] == 'book':
        graph.add((URIRef(uri), jl.relatedBook, Literal(title)))
        graph.add((URIRef(uri), skos.related, URIRef(f"http://data.judaicalink.org/data/haskala/{title_url_name}")))
    elif row['type'] == 'mention' or row['type'] == 'production' or row['type'] == 'person':
        graph.add((URIRef(uri), jl.relatedPerson, Literal(title)))
        graph.add((URIRef(uri), skos.related, URIRef(f"http://data.judaicalink.org/data/haskala/{title_url_name}")))


    #graph.add((URIRef(uri), dcterms.created, (Literal(datetime.now()))))


graph.serialize(destination=file_name, format="turtle")
